In [ ]:
import cellxgene_census as cc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


class adata:
    def__init__(self, X=None, obs=None, var=None, uns=None):
        self.X = X
        self.obs = obs
        self.var = var
        self.uns = uns  = uns if uns is not None else {}


function cellxgene_query(dataset_id, filters=None):

    # Simulated function to query cellxgene census datasets

    return adata()

# Load dataset
dataset_id = "example_dataset_id"
adata = cc.load_dataset(dataset_id)

# Display basic information about the dataset
print(adata)
print(f"Number of cells: {adata.X.shape[0]}")
print(f"Number of genes: {adata.X.shape[1]}")





In [1]:
from pathlib import Path
import cellxgene_census as cxc
import pyarrow.parquet as pq

OUT = Path("census_hsapiens_healthy_adult_obs.parquet")

# Replace these with values discovered from summary_cell_counts
HEALTHY_DISEASE_OTID = "REPLACE_ME"
ADULT_STAGE_OTID = "REPLACE_ME"

# Keep columns minimal for speed and file size
OBS_COLS = [
    "dataset_id",
    "donor_id",
    "assay",
    "sex",
    "tissue",
    "tissue_general",
    "cell_type",
    "disease",
    "development_stage",
    "is_primary_data",
    # optionally include ontology IDs if you want them in the output
    "tissue_ontology_term_id",
    "tissue_general_ontology_term_id",
    "cell_type_ontology_term_id",
    "disease_ontology_term_id",
    "development_stage_ontology_term_id",
]

VALUE_FILTER = (
    "is_primary_data == True and "
    f"disease_ontology_term_id == '{HEALTHY_DISEASE_OTID}' and "
    f"development_stage_ontology_term_id == '{ADULT_STAGE_OTID}'"
)

with cxc.open_soma(census_version="stable") as census:
    human = census["census_data"]["homo_sapiens"]

    read_iter = human.obs.read(
        value_filter=VALUE_FILTER,
        column_names=OBS_COLS,
        # batch_size is supported by SOMA DataFrame reads (iterator over Arrow tables)
        batch_size=200_000,
    )

    writer = None
    try:
        for i, tbl in enumerate(read_iter):
            if writer is None:
                writer = pq.ParquetWriter(OUT.as_posix(), tbl.schema)
            writer.write_table(tbl)
            if (i + 1) % 10 == 0:
                print(f"Wrote {i+1} batches...")
    finally:
        if writer is not None:
            writer.close()

print(f"Done: {OUT}")


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


PermissionError: [Errno 13] Failed to open local file 'census_hsapiens_healthy_adult_obs.parquet'. Detail: [errno 13] Permission denied

In [4]:
"""
Export CELLxGENE Census human (homo_sapiens) OBS metadata for healthy adults
across all tissues to a Parquet file, streaming in batches to avoid OOM.

Key features:
- Writes to a user-writable output directory (HOME/census_exports by default)
- Validates writability before starting
- Streams Arrow record batches -> Parquet (single file)
- Minimal, explicit column selection
- Clear logging and failure modes

Prereqs:
  pip install -U cellxgene-census pyarrow pandas
"""

from __future__ import annotations

import os
from pathlib import Path
from typing import Iterable, Optional, List, Tuple

import pyarrow as pa
import pyarrow.parquet as pq
import cellxgene_census as cxc


# ----------------------------
# User-configurable parameters
# ----------------------------

CENSUS_VERSION = "stable"
ORGANISM = "homo_sapiens"

# Replace these two with the ontology IDs you identified from summary_cell_counts
HEALTHY_DISEASE_OTID = "REPLACE_ME"  # e.g., "PATO:0000461" (example only; use your discovered value)
ADULT_STAGE_OTID = "REPLACE_ME"      # e.g., "HsapDv:0000087" (example only; use your discovered value)

# Output settings
OUTDIR = Path(os.environ.get("CENSUS_OUTDIR", str(Path.home() / "census_exports")))
OUTFILE = "census_hsapiens_healthy_adult_obs.parquet"
OVERWRITE = True

# Streaming settings
BATCH_SIZE = 200_000  # tune up/down depending on RAM and IO
LOG_EVERY_N_BATCHES = 10

# Minimal useful OBS columns (add/remove as needed)
OBS_COLS: List[str] = [
    "dataset_id",
    "donor_id",
    "assay",
    "sex",
    "tissue",
    "tissue_general",
    "cell_type",
    "disease",
    "development_stage",
    "is_primary_data",
    "tissue_ontology_term_id",
    "tissue_general_ontology_term_id",
    "cell_type_ontology_term_id",
    "disease_ontology_term_id",
    "development_stage_ontology_term_id",
]


# ----------------------------
# Helper functions
# ----------------------------

def ensure_writable_dir(outdir: Path) -> None:
    """Create outdir (if needed) and ensure we can write to it."""
    outdir.mkdir(parents=True, exist_ok=True)
    if not os.access(outdir, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {outdir}")


def resolve_output_path(outdir: Path, filename: str, overwrite: bool) -> Path:
    """Resolve output path and handle overwrite behavior."""
    ensure_writable_dir(outdir)
    outpath = outdir / filename
    if outpath.exists():
        if overwrite:
            # Ensure we can remove the existing file
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file (not writable): {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output file already exists: {outpath}")
    return outpath


def build_value_filter(
    healthy_disease_otid: str,
    adult_stage_otid: str,
    require_primary: bool = True,
) -> str:
    """Build SOMA value_filter for OBS selection."""
    parts = []
    if require_primary:
        parts.append("is_primary_data == True")
    parts.append(f"disease_ontology_term_id == '{healthy_disease_otid}'")
    parts.append(f"development_stage_ontology_term_id == '{adult_stage_otid}'")
    return " and ".join(parts)


def stream_obs_arrow_tables(
    census: cxc.SOMACollection,
    organism: str,
    value_filter: str,
    columns: List[str],
    batch_size: int,
) -> Iterable[pa.Table]:
    """Yield Arrow tables from Census OBS with a filter, streaming by batch_size."""
    exp = census["census_data"][organism]
    reader = exp.obs.read(
        value_filter=value_filter,
        column_names=columns,
        batch_size=batch_size,
    )
    for tbl in reader:
        # Defensive: ensure it's a Table (some Arrow flows can return RecordBatch)
        yield tbl if isinstance(tbl, pa.Table) else pa.Table.from_batches([tbl])


def write_parquet_stream(
    tables: Iterable[pa.Table],
    outpath: Path,
    log_every: int = 10,
) -> Tuple[int, int]:
    """
    Write an iterable of Arrow tables to a single Parquet file.

    Returns:
      (num_batches_written, total_rows_written)
    """
    writer: Optional[pq.ParquetWriter] = None
    n_batches = 0
    n_rows = 0

    try:
        for i, tbl in enumerate(tables, start=1):
            if tbl.num_rows == 0:
                continue

            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema)

            writer.write_table(tbl)
            n_batches += 1
            n_rows += tbl.num_rows

            if log_every and (i % log_every == 0):
                print(f"[write] batches seen={i:,}  batches_written={n_batches:,}  rows_written={n_rows:,}")

    finally:
        if writer is not None:
            writer.close()

    return n_batches, n_rows


def validate_placeholders(healthy_disease_otid: str, adult_stage_otid: str) -> None:
    """Fail fast if the user forgot to replace placeholder ontology IDs."""
    for name, val in [("HEALTHY_DISEASE_OTID", healthy_disease_otid), ("ADULT_STAGE_OTID", adult_stage_otid)]:
        if (val is None) or (not isinstance(val, str)) or (val.strip() == "") or ("REPLACE_ME" in val):
            raise ValueError(
                f"{name} is not set. Replace the placeholder with the ontology_term_id you discovered "
                f"from census_info/summary_cell_counts."
            )


# ----------------------------
# Main
# ----------------------------

def main() -> None:
    validate_placeholders(HEALTHY_DISEASE_OTID, ADULT_STAGE_OTID)

    outpath = resolve_output_path(OUTDIR, OUTFILE, OVERWRITE)
    print("CWD:", Path.cwd())
    print("Output:", outpath)

    value_filter = build_value_filter(
        healthy_disease_otid=HEALTHY_DISEASE_OTID,
        adult_stage_otid=ADULT_STAGE_OTID,
        require_primary=True,
    )
    print("OBS filter:", value_filter)

    with cxc.open_soma(census_version=CENSUS_VERSION) as census:
        tables = stream_obs_arrow_tables(
            census=census,
            organism=ORGANISM,
            value_filter=value_filter,
            columns=OBS_COLS,
            batch_size=BATCH_SIZE,
        )
        n_batches, n_rows = write_parquet_stream(
            tables=tables,
            outpath=outpath,
            log_every=LOG_EVERY_N_BATCHES,
        )

    print(f"Done. Wrote {n_rows:,} rows across {n_batches:,} batches to:\n  {outpath}")


if __name__ == "__main__":
    main()


ValueError: HEALTHY_DISEASE_OTID is not set. Replace the placeholder with the ontology_term_id you discovered from census_info/summary_cell_counts.

In [5]:
import cellxgene_census as cxc

with cxc.open_soma(census_version="stable") as census:
    scc = census["census_info"]["summary_cell_counts"].read().concat().to_pandas()

hs = scc[scc["organism"].astype(str).str.contains("sapiens", case=False, na=False)]
subset = hs[hs["category"].isin(["disease", "development_stage"])].copy()

# Show the most common labels to help you choose the correct IDs
print(
    subset.sort_values(["category", "unique_cell_count"], ascending=[True, False])
          .loc[:, ["category", "label", "ontology_term_id", "unique_cell_count"]]
          .head(60)
)


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


     category                                              label  \
1284  disease                                             normal   
1171  disease                          cytomegalovirus infection   
1274  disease                                           COVID-19   
1177  disease                                  Parkinson disease   
1041  disease                      dementia || Alzheimer disease   
1030  disease                                           dementia   
1249  disease                                       glioblastoma   
1214  disease                                      breast cancer   
1270  disease                     malignant ovarian serous tumor   
1145  disease                                lung adenocarcinoma   
1219  disease                       systemic lupus erythematosus   
1241  disease                        B-cell non-Hodgkin lymphoma   
1122  disease                                  Alzheimer disease   
1138  disease                             dilate

In [6]:
from __future__ import annotations

import os
from pathlib import Path
from typing import List, Tuple, Optional

import pyarrow as pa
import pyarrow.parquet as pq
import cellxgene_census as cxc


# -----------------------------
# Configuration
# -----------------------------
CENSUS_VERSION = "stable"
ORGANISM = "homo_sapiens"

# From your output: disease "normal" => PATO:0000461
HEALTHY_DISEASE_OTID = "PATO:0000461"

# Output location (override with env var if desired)
OUTDIR = Path(os.environ.get("CENSUS_OUTDIR", str(Path.home() / "census_exports")))
OUTFILE = "census_hsapiens_healthy_adult_obs.parquet"
OVERWRITE = True

# Streaming parameters
BATCH_SIZE = 200_000
LOG_EVERY_N = 10

# Keep columns minimal to reduce output size
OBS_COLS: List[str] = [
    "dataset_id",
    "donor_id",
    "assay",
    "sex",
    "tissue",
    "tissue_general",
    "cell_type",
    "disease",
    "development_stage",
    "is_primary_data",
    "tissue_ontology_term_id",
    "tissue_general_ontology_term_id",
    "cell_type_ontology_term_id",
    "disease_ontology_term_id",
    "development_stage_ontology_term_id",
]


# -----------------------------
# Helpers
# -----------------------------
def ensure_writable_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path}")


def resolve_outpath(outdir: Path, outfile: str, overwrite: bool) -> Path:
    ensure_writable_dir(outdir)
    outpath = outdir / outfile
    if outpath.exists():
        if overwrite:
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite (no write permission): {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output exists and OVERWRITE=False: {outpath}")
    return outpath


def discover_adult_stage_otids(census: cxc.SOMACollection) -> List[Tuple[str, str, int]]:
    """
    Returns list of (label, ontology_term_id, unique_cell_count) for human development_stage
    entries whose label contains 'adult' (case-insensitive).
    """
    scc = census["census_info"]["summary_cell_counts"].read().concat().to_pandas()

    hs = scc[
        scc["organism"].astype(str).str.contains("sapiens", case=False, na=False)
        & (scc["category"] == "development_stage")
    ].copy()

    hs["label_str"] = hs["label"].astype(str)
    adult = hs[hs["label_str"].str.contains("adult", case=False, na=False)].copy()

    adult = adult.sort_values("unique_cell_count", ascending=False)
    results = list(
        zip(
            adult["label"].astype(str).tolist(),
            adult["ontology_term_id"].astype(str).tolist(),
            adult["unique_cell_count"].astype(int).tolist(),
        )
    )
    return results


def or_chain_equals(field: str, values: List[str]) -> str:
    """
    Build a safe OR-chain expression:
      (field == 'v1' or field == 'v2' or ...)
    """
    if not values:
        return ""
    parts = [f"{field} == '{v}'" for v in values]
    return "(" + " or ".join(parts) + ")"


def build_value_filter(healthy_disease_otid: str, adult_stage_otids: List[str]) -> str:
    base = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{healthy_disease_otid}'",
    ]

    adult_expr = or_chain_equals("development_stage_ontology_term_id", adult_stage_otids)
    if adult_expr:
        base.append(adult_expr)

    return " and ".join(base)


def export_obs_to_parquet(
    census: cxc.SOMACollection,
    organism: str,
    value_filter: str,
    obs_cols: List[str],
    outpath: Path,
    batch_size: int,
    log_every_n: int,
) -> Tuple[int, int]:
    """
    Streams OBS rows matching value_filter to a single Parquet file.
    Returns (batches_written, rows_written).
    """
    exp = census["census_data"][organism]
    reader = exp.obs.read(
        value_filter=value_filter,
        column_names=obs_cols,
        batch_size=batch_size,
    )

    writer: Optional[pq.ParquetWriter] = None
    batches_written = 0
    rows_written = 0

    try:
        for i, tbl in enumerate(reader, start=1):
            if not isinstance(tbl, pa.Table):
                tbl = pa.Table.from_batches([tbl])

            if tbl.num_rows == 0:
                continue

            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema)

            writer.write_table(tbl)
            batches_written += 1
            rows_written += tbl.num_rows

            if log_every_n and (i % log_every_n == 0):
                print(f"[export] batches_seen={i:,} batches_written={batches_written:,} rows_written={rows_written:,}")

    finally:
        if writer is not None:
            writer.close()

    return batches_written, rows_written


# -----------------------------
# Main execution
# -----------------------------
def main() -> None:
    outpath = resolve_outpath(OUTDIR, OUTFILE, OVERWRITE)
    print("CWD:", Path.cwd())
    print("Output:", outpath)

    with cxc.open_soma(census_version=CENSUS_VERSION) as census:
        adult_entries = discover_adult_stage_otids(census)

        if not adult_entries:
            print("WARNING: No development_stage labels containing 'adult' were found in summary_cell_counts.")
            print("         The export will proceed WITHOUT a development_stage filter (healthy only).")
            adult_stage_otids = []
        else:
            print("Discovered adult development_stage candidates (label, ontology_term_id, unique_cell_count):")
            for label, otid, n in adult_entries[:25]:
                print(f"  - {label} | {otid} | {n:,}")
            adult_stage_otids = [otid for _, otid, _ in adult_entries]

        value_filter = build_value_filter(HEALTHY_DISEASE_OTID, adult_stage_otids)
        print("\nOBS value_filter:")
        print(value_filter)
        print()

        batches, rows = export_obs_to_parquet(
            census=census,
            organism=ORGANISM,
            value_filter=value_filter,
            obs_cols=OBS_COLS,
            outpath=outpath,
            batch_size=BATCH_SIZE,
            log_every_n=LOG_EVERY_N,
        )

    print(f"Done. Wrote {rows:,} rows across {batches:,} batches to:\n  {outpath}")


if __name__ == "__main__":
    main()


CWD: /home
Output: /home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


         The export will proceed WITHOUT a development_stage filter (healthy only).

OBS value_filter:
is_primary_data == True and disease_ontology_term_id == 'PATO:0000461'

Done. Wrote 62,464,283 rows across 1 batches to:
  /home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet


In [7]:
from __future__ import annotations

import os
import re
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import cellxgene_census
import tiledbsoma

# --- BatchSize import (location varies by tiledbsoma version) ---
try:
    from tiledbsoma.options import BatchSize
except ImportError:
    from tiledbsoma import BatchSize


# ----------------------------
# User-configurable parameters
# ----------------------------
CENSUS_VERSION = "stable"  # or a pinned date like "2025-11-08"
ORGANISM = "homo_sapiens"

TARGET_ASSAY_LABELS = ["10x 3' v2", "10x 3' v3", "10x multiome"]

# ontology ID for "normal" disease in the Census (you already used this successfully)
NORMAL_DISEASE_ONTOLOGY_ID = "PATO:0000461"

SUSPENSION_TYPE = "cell"
SEXES = ["male", "female"]
MIN_AGE_YEARS = 15

# output
OUT_DIR = Path.home() / "census_exports" / "hsapiens_normal_10x_adult15_cell_obs_parts"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# how many rows per parquet part (target; exact size may vary)
ROWS_PER_BATCH = 250_000


def _format_in_list(values: list[str]) -> str:
    """Return a SOMA value_filter-compatible list literal: ['a','b']"""
    escaped = [v.replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in escaped]) + "]"


def lookup_assay_term_ids(census) -> dict[str, str]:
    """
    Returns mapping: assay_label -> assay_ontology_term_id
    using census_info/summary_cell_counts.
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )

    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(TARGET_ASSAY_LABELS))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in TARGET_ASSAY_LABELS if x not in found]
    if missing:
        raise ValueError(
            f"Did not find ontology_term_id for assay label(s): {missing}. "
            f"Found={list(found.keys())}"
        )
    return found


def infer_age_filter_expr(obs_schema: pa.Schema) -> str | None:
    """
    Prefer in-query filter if donor_age is numeric and (optionally) donor_age_unit exists.
    Otherwise return None (caller can post-filter).
    """
    fields = {f.name: f.type for f in obs_schema}

    if "donor_age" not in fields:
        return None

    donor_age_type = fields["donor_age"]
    donor_age_is_numeric = pa.types.is_integer(donor_age_type) or pa.types.is_floating(donor_age_type)
    if not donor_age_is_numeric:
        return None

    if "donor_age_unit" in fields:
        # Most robust: enforce years
        return f"(donor_age >= {MIN_AGE_YEARS} and donor_age_unit == 'year')"
    else:
        return f"(donor_age >= {MIN_AGE_YEARS})"


_age_re = re.compile(r"(\d+(?:\.\d+)?)")


def postfilter_age_if_needed(df: pd.DataFrame) -> pd.DataFrame:
    """
    If the query couldn't filter by age in the backend, do a best-effort ≥15y filter here.
    Handles:
      - numeric donor_age (+ optional donor_age_unit)
      - string donor_age like '34 years'
    Rows with unknown/invalid age are dropped.
    """
    if "donor_age" not in df.columns:
        # Nothing we can do; return unchanged.
        return df

    # If donor_age is numeric already:
    if pd.api.types.is_numeric_dtype(df["donor_age"]):
        if "donor_age_unit" in df.columns:
            df = df[df["donor_age_unit"] == "year"]
        return df[df["donor_age"] >= MIN_AGE_YEARS]

    # Otherwise try parsing from strings
    donor_age_str = df["donor_age"].astype("string")
    parsed = donor_age_str.str.extract(_age_re, expand=False)
    parsed_num = pd.to_numeric(parsed, errors="coerce")

    # enforce years if unit exists OR if string contains 'year'
    if "donor_age_unit" in df.columns:
        unit_ok = df["donor_age_unit"].astype("string") == "year"
    else:
        unit_ok = donor_age_str.str.contains("year", case=False, na=False)

    keep = unit_ok & (parsed_num >= MIN_AGE_YEARS)
    return df.loc[keep].copy()


def main():
    print(f"Writing Parquet parts to:\n  {OUT_DIR}")

    with cellxgene_census.open_soma(census_version=CENSUS_VERSION) as census:
        human = census["census_data"][ORGANISM]

        # 1) Resolve assay ontology IDs (avoids apostrophe issues in assay labels)
        assay_map = lookup_assay_term_ids(census)
        assay_term_ids = [assay_map[x] for x in TARGET_ASSAY_LABELS]
        print("Assay ontology_term_id mapping:")
        for k, v in assay_map.items():
            print(f"  {k:12s} -> {v}")

        # 2) Build the base filter
        base_filter_parts = [
            "is_primary_data == True",
            f"disease_ontology_term_id == '{NORMAL_DISEASE_ONTOLOGY_ID}'",
            f"sex in {_format_in_list(SEXES)}",
            f"suspension_type == '{SUSPENSION_TYPE}'",
            f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}",
        ]

        # 3) Add in-backend age filter if possible
        obs_schema = human.obs.schema
        age_expr = infer_age_filter_expr(obs_schema)
        if age_expr is None:
            print(
                "WARNING: Could not apply an in-query age filter (>=15y). "
                "Will post-filter age per batch after reading."
            )
        else:
            base_filter_parts.append(age_expr)

        obs_value_filter = " and ".join(base_filter_parts)
        print("\nOBS value_filter:\n", obs_value_filter)

        # 4) Optional: count how many cells match without reading all metadata
        # (axis_query + n_obs is the standard way to get the filtered size) :contentReference[oaicite:5]{index=5}
        query = human.axis_query(
            measurement_name="RNA",
            obs_query=tiledbsoma.AxisQuery(value_filter=obs_value_filter),
        )
        try:
            print(f"\nFiltered cell count (query.n_obs): {query.n_obs:,}")
        finally:
            query.close()

        # 5) Stream obs rows and write parquet parts
        batch_size = BatchSize(count=ROWS_PER_BATCH)
        read_iter = human.obs.read(value_filter=obs_value_filter, batch_size=batch_size)

        total_rows_written = 0
        part_idx = 0

        for tbl in read_iter:
            # If we couldn't do in-query age filter, apply post-filter here
            if age_expr is None:
                df = tbl.to_pandas()
                df = postfilter_age_if_needed(df)
                if df.empty:
                    continue
                tbl = pa.Table.from_pandas(df, preserve_index=False)

            out_path = OUT_DIR / f"part-{part_idx:05d}.parquet"
            pq.write_table(tbl, out_path, compression="zstd")

            n = tbl.num_rows
            total_rows_written += n
            if part_idx % 10 == 0:
                print(f"  wrote {out_path.name}: {n:,} rows (running total {total_rows_written:,})")

            part_idx += 1

        print(f"\nDone. Wrote {total_rows_written:,} rows across {part_idx} part files.")
        print(f"Output directory:\n  {OUT_DIR}")


if __name__ == "__main__":
    main()


ImportError: cannot import name 'BatchSize' from 'tiledbsoma' (/home/crizza/miniconda3/lib/python3.12/site-packages/tiledbsoma/__init__.py)

In [8]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq

import cellxgene_census as cxc


# -----------------------------
# User configuration
# -----------------------------
CENSUS_VERSION = "stable"
ORGANISM = "homo_sapiens"

# Filters requested
NORMAL_DISEASE_OTID = "PATO:0000461"  # "normal"
TARGET_ASSAY_LABELS = ["10x 3' v2", "10x 3' v3", "10x multiome"]
TARGET_SEXES = ["male", "female"]
TARGET_SUSPENSION_TYPE = "cell"
MIN_AGE_YEARS = 15

# Output
OUTDIR = Path(os.environ.get("CENSUS_OUTDIR", str(Path.home() / "census_exports")))
OUTFILE = "census_hsapiens_normal_10x_adult15_cell_obs.parquet"
OVERWRITE = True

# Keep columns tight to reduce file size
OBS_COLS: List[str] = [
    "dataset_id",
    "donor_id",
    "assay",
    "assay_ontology_term_id",
    "sex",
    "suspension_type",
    "disease",
    "disease_ontology_term_id",
    "development_stage",
    "development_stage_ontology_term_id",
    "donor_age",
    "donor_age_unit",
    "tissue",
    "tissue_general",
    "cell_type",
    "is_primary_data",
]


# -----------------------------
# Helpers
# -----------------------------
def ensure_writable_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path}")


def resolve_outpath(outdir: Path, outfile: str, overwrite: bool) -> Path:
    ensure_writable_dir(outdir)
    outpath = outdir / outfile
    if outpath.exists():
        if overwrite:
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output exists and OVERWRITE=False: {outpath}")
    return outpath


def _format_in_list(values: List[str]) -> str:
    # SOMA value_filter list literal: ['a','b']
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def lookup_assay_term_ids(census) -> Dict[str, str]:
    """
    Map assay label -> assay_ontology_term_id using census_info/summary_cell_counts.
    This avoids quoting problems with "10x 3' v3".
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )
    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(TARGET_ASSAY_LABELS))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in TARGET_ASSAY_LABELS if x not in found]
    if missing:
        raise ValueError(
            f"Did not find ontology_term_id for assay label(s): {missing}. "
            f"Found: {list(found.keys())}"
        )
    return found


def has_field(schema: pa.Schema, name: str) -> bool:
    return name in schema.names


def build_obs_value_filter(
    assay_term_ids: Optional[List[str]],
    obs_schema: pa.Schema,
) -> Tuple[str, bool]:
    """
    Returns (value_filter, needs_post_age_filter).
    We try to express age>=15 years in the backend if donor_age is numeric.
    Otherwise we will post-filter by age on Arrow tables.
    """
    parts = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{NORMAL_DISEASE_OTID}'",
        f"sex in {_format_in_list(TARGET_SEXES)}",
        f"suspension_type == '{TARGET_SUSPENSION_TYPE}'",
    ]

    if assay_term_ids is not None and has_field(obs_schema, "assay_ontology_term_id"):
        parts.append(f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}")
    else:
        # Fallback to label filter if ontology field is absent
        parts.append(f"assay in {_format_in_list(TARGET_ASSAY_LABELS)}")

    # Age handling
    needs_post_age = True
    if has_field(obs_schema, "donor_age") and pa.types.is_numeric(obs_schema.field("donor_age").type):
        if has_field(obs_schema, "donor_age_unit"):
            parts.append(f"(donor_age >= {MIN_AGE_YEARS} and donor_age_unit == 'year')")
        else:
            # If no unit, assume years (best-effort; verify if needed)
            parts.append(f"(donor_age >= {MIN_AGE_YEARS})")
        needs_post_age = False

    return " and ".join(parts), needs_post_age


def post_filter_adult_15_plus(tbl: pa.Table) -> pa.Table:
    """
    Post-filter to adults (>=15y) when we could not express age filter in value_filter.
    Priority:
      1) donor_age numeric (+ donor_age_unit == 'year' when present)
      2) development_stage_ontology_term_id adult-stage heuristic (coarser)
    """
    schema = tbl.schema

    # 1) donor_age path
    if "donor_age" in schema.names:
        col = tbl["donor_age"]
        # numeric donor_age
        if pa.types.is_numeric(col.type):
            mask = pc.greater_equal(col, pa.scalar(MIN_AGE_YEARS, type=col.type))
            if "donor_age_unit" in schema.names:
                mask = pc.and_(mask, pc.equal(tbl["donor_age_unit"], pa.scalar("year")))
            return tbl.filter(mask)

    # 2) fallback: development_stage ontology heuristic (coarser than real age)
    # Adult stage root in HsapDv is commonly HsapDv:0000087 (and possible descendants).
    # This is a best-effort fallback ONLY if donor_age is not usable.
    if "development_stage_ontology_term_id" in schema.names:
        stage = tbl["development_stage_ontology_term_id"]
        # startswith("HsapDv:0000087") OR equals common adult terms
        # (Some datasets use the root directly, others use children; this remains heuristic.)
        adult_terms = pa.array(
            ["HsapDv:0000087", "HsapDv:0000088", "HsapDv:0000089"],
            type=pa.string(),
        )
        mask = pc.is_in(stage, value_set=adult_terms)
        return tbl.filter(mask)

    # If neither exists, return unchanged (cannot enforce adult criterion)
    return tbl


def enforce_columns(tbl: pa.Table, desired: List[str]) -> pa.Table:
    cols = [c for c in desired if c in tbl.schema.names]
    return tbl.select(cols)


# -----------------------------
# Main
# -----------------------------
def main() -> None:
    outpath = resolve_outpath(OUTDIR, OUTFILE, OVERWRITE)
    print("Output:", outpath)

    with cxc.open_soma(census_version=CENSUS_VERSION) as census:
        human = census["census_data"][ORGANISM]
        obs_schema = human.obs.schema

        # Resolve assay ontology IDs (preferred)
        assay_term_ids: Optional[List[str]] = None
        try:
            assay_map = lookup_assay_term_ids(census)
            assay_term_ids = [assay_map[x] for x in TARGET_ASSAY_LABELS]
            print("Assay term IDs:")
            for k, v in assay_map.items():
                print(f"  {k} -> {v}")
        except Exception as e:
            print(f"WARNING: Could not resolve assay_ontology_term_id mapping; falling back to label filter. Reason: {e}")

        value_filter, needs_post_age = build_obs_value_filter(assay_term_ids, obs_schema)
        print("\nOBS value_filter:\n", value_filter)
        if needs_post_age:
            print("NOTE: donor_age is not a numeric field in this obs schema; applying adult>=15y as a post-filter.\n")

        # Read iterator (no BatchSize required)
        read_iter = human.obs.read(value_filter=value_filter, column_names=OBS_COLS)

        writer: Optional[pq.ParquetWriter] = None
        rows_written = 0
        batches_seen = 0

        try:
            for tbl in read_iter:
                batches_seen += 1
                if not isinstance(tbl, pa.Table):
                    tbl = pa.Table.from_batches([tbl])

                # Apply post-filter for age if needed
                if needs_post_age:
                    tbl = post_filter_adult_15_plus(tbl)

                # Ensure consistent column set (in case some are missing)
                tbl = enforce_columns(tbl, OBS_COLS)

                if tbl.num_rows == 0:
                    continue

                if writer is None:
                    writer = pq.ParquetWriter(str(outpath), tbl.schema, compression="zstd")

                writer.write_table(tbl)
                rows_written += tbl.num_rows

                if batches_seen % 5 == 0:
                    print(f"[progress] batches_seen={batches_seen:,} rows_written={rows_written:,}")

        finally:
            if writer is not None:
                writer.close()

        print(f"\nDone. Wrote {rows_written:,} rows to:\n  {outpath}")


if __name__ == "__main__":
    main()


Output: /home/crizza/census_exports/census_hsapiens_normal_10x_adult15_cell_obs.parquet


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


Assay term IDs:
  10x 3' v3 -> EFO:0009922
  10x 3' v2 -> EFO:0009899
  10x multiome -> EFO:0030059

OBS value_filter:
 is_primary_data == True and disease_ontology_term_id == 'PATO:0000461' and sex in ['male', 'female'] and suspension_type == 'cell' and assay_ontology_term_id in ['EFO:0009899', 'EFO:0009922', 'EFO:0030059']
NOTE: donor_age is not a numeric field in this obs schema; applying adult>=15y as a post-filter.



SOMAError: [SOMAArray] internal coding error: No column named donor_age found

In [9]:
import pyarrow.parquet as pq
import pandas as pd

p = "/home/crizza/census_exports/census_hsapiens_normal_10x_adult15_cell_obs.parquet"
t = pq.read_table(p, columns=[c for c in ["donor_age", "donor_age_unit", "development_stage"] if c is not None])
df = t.to_pandas()

print(df[["donor_age","donor_age_unit"]].dropna().describe(include="all"))
print(df["development_stage"].value_counts().head(20))


FileNotFoundError: /home/crizza/census_exports/census_hsapiens_normal_10x_adult15_cell_obs.parquet

In [13]:
from collections import Counter
from pathlib import Path

import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq

P = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")

def _col(batch: pa.RecordBatch, name: str):
    idx = batch.schema.get_field_index(name)
    return None if idx == -1 else batch.column(idx)

def _update_counter_from_value_counts(counter: Counter, arr: pa.Array):
    # Cast to string to avoid dictionary-type edge cases
    arr = pc.cast(arr, pa.string())
    vc = pc.value_counts(arr)  # struct array with fields: values, counts
    vals = vc.field("values").to_pylist()
    cts  = vc.field("counts").to_pylist()
    for v, c in zip(vals, cts):
        if v is None:
            continue
        counter[v] += int(c)

def scan_dev_stage(parquet_path: Path, batch_size=250_000, max_rows=2_000_000):
    pf = pq.ParquetFile(parquet_path)

    cols = [
        "development_stage",
        "development_stage_ontology_term_id",
        "donor_age",
        "donor_age_unit",
    ]
    available = set(pf.schema_arrow.names)
    cols = [c for c in cols if c in available]

    if "development_stage" not in cols:
        raise ValueError("development_stage column not found in the parquet.")

    stage_counts = Counter()
    stage_otid_counts = Counter()
    pair_counts = Counter()
    adult_like_counts = Counter()

    rows_scanned = 0
    looks_like_ontid = 0

    for batch in pf.iter_batches(columns=cols, batch_size=batch_size):
        # Stop after max_rows (sample is usually enough to infer encoding)
        if rows_scanned >= max_rows:
            break

        if rows_scanned + batch.num_rows > max_rows:
            batch = batch.slice(0, max_rows - rows_scanned)

        rows_scanned += batch.num_rows

        stage = _col(batch, "development_stage")
        otid  = _col(batch, "development_stage_ontology_term_id")

        stage_str = pc.cast(stage, pa.string())

        # 1) top development_stage
        _update_counter_from_value_counts(stage_counts, stage_str)

        # 2) does development_stage look like ontology IDs?
        ont_mask = pc.match_substring_regex(stage_str, r"^[A-Za-z]+:[0-9]+$")
        looks_like_ontid += int(pc.sum(pc.cast(ont_mask, pa.int64())).as_py())

        # 3) any English-like "adult" labels?
        adult_mask = pc.match_substring(pc.utf8_lower(stage_str), "adult")
        adult_vals = pc.filter(stage_str, adult_mask)
        if len(adult_vals) > 0:
            _update_counter_from_value_counts(adult_like_counts, adult_vals)

        # 4) stage ontology IDs + (stage, otid) pairs
        if otid is not None:
            otid_str = pc.cast(otid, pa.string())
            _update_counter_from_value_counts(stage_otid_counts, otid_str)

            # Pair counting without struct value_counts:
            # Convert just this batch to Python lists (bounded by max_rows sample)
            stage_list = stage_str.to_pylist()
            otid_list  = otid_str.to_pylist()

            for s, o in zip(stage_list, otid_list):
                if s is None and o is None:
                    continue
                pair_counts[(s, o)] += 1

    return {
        "rows_scanned": rows_scanned,
        "pct_stage_looks_like_ontid": 100.0 * looks_like_ontid / max(rows_scanned, 1),
        "stage_counts": stage_counts,
        "stage_otid_counts": stage_otid_counts,
        "pair_counts": pair_counts,
        "adult_like_counts": adult_like_counts,
        "columns_present": cols,
    }

res = scan_dev_stage(P, batch_size=250_000, max_rows=2_000_000)

print("Columns present:", res["columns_present"])
print("Rows scanned:", f"{res['rows_scanned']:,}")
print("Pct development_stage that looks like ontology ID:", f"{res['pct_stage_looks_like_ontid']:.2f}%")

print("\nTop development_stage:")
for k, v in res["stage_counts"].most_common(25):
    print(f"  {k} -> {v:,}")

print("\nTop development_stage_ontology_term_id:")
for k, v in res["stage_otid_counts"].most_common(25):
    print(f"  {k} -> {v:,}")

print("\nTop (development_stage, development_stage_ontology_term_id) pairs:")
for (s, o), n in res["pair_counts"].most_common(25):
    print(f"  ({s}, {o}) -> {n:,}")

print("\nStages containing 'adult' (if stored as English labels):")
for k, v in res["adult_like_counts"].most_common(25):
    print(f"  {k} -> {v:,}")


Columns present: ['development_stage', 'development_stage_ontology_term_id']
Rows scanned: 2,000,000
Pct development_stage that looks like ontology ID: 0.00%

Top development_stage:
  50-year-old stage -> 304,688
  42-year-old stage -> 196,204
  29-year-old stage -> 165,224
  22nd week post-fertilization stage -> 118,647
  adult stage -> 115,964
  seventh decade stage -> 66,740
  7-month-old stage -> 51,878
  26-year-old stage -> 49,772
  69-year-old stage -> 43,764
  60-year-old stage -> 36,428
  young adult stage -> 30,750
  63-year-old stage -> 30,708
  54-year-old stage -> 29,585
  unknown -> 28,634
  prime adult stage -> 27,864
  sixth decade stage -> 25,501
  10th week post-fertilization stage -> 25,358
  62-year-old stage -> 25,070
  3-year-old stage -> 24,917
  65-year-old stage -> 24,366
  middle aged stage -> 24,222
  71-year-old stage -> 23,847
  59-year-old stage -> 23,336
  sixth LMP month stage -> 20,475
  53-year-old stage -> 19,407

Top development_stage_ontology_term_i

In [16]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Optional

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

INP = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")
OUT = Path("/home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_adult15.parquet")

TARGET_ASSAYS = {"10x 3' v2", "10x 3' v3", "10x multiome"}
TARGET_SEXES  = {"male", "female"}
TARGET_SUSP   = "cell"

# normal disease in Census:
NORMAL_DISEASE_OTID = "PATO:0000461"
NORMAL_DISEASE_LABEL = "normal"

BATCH_SIZE = 250_000  # adjust up/down depending on memory


def require_columns(pf: pq.ParquetFile, required: list[str]) -> None:
    missing = [c for c in required if c not in pf.schema_arrow.names]
    if missing:
        raise ValueError(
            "Missing required column(s) in the Parquet file: "
            + ", ".join(missing)
            + "\nRe-export the Census obs including these fields, or adjust the filter to available columns."
        )


def discover_adult_labels_and_ids(pf: pq.ParquetFile) -> pd.DataFrame:
    """
    Scan development_stage + ontology id and return unique pairs where stage contains 'adult'.
    """
    cols = ["development_stage", "development_stage_ontology_term_id"]
    have_otid = "development_stage_ontology_term_id" in pf.schema_arrow.names
    if not have_otid:
        cols = ["development_stage"]

    rows = []
    for batch in pf.iter_batches(columns=cols, batch_size=BATCH_SIZE):
        df = batch.to_pandas()
        s = df["development_stage"].astype("string")
        m = s.str.contains("adult", case=False, na=False)
        if not m.any():
            continue
        if have_otid:
            rows.append(df.loc[m, ["development_stage", "development_stage_ontology_term_id"]])
        else:
            rows.append(df.loc[m, ["development_stage"]])

    if not rows:
        return pd.DataFrame(columns=cols)

    out = pd.concat(rows, ignore_index=True).drop_duplicates()
    return out.sort_values(cols[0]).reset_index(drop=True)


def adult15_mask(stage: pd.Series) -> pd.Series:
    """
    Adults (>=15 years) inferred from development_stage strings.
    Includes:
      - NN-year-old stage where NN>=15
      - adult/young adult/prime adult/late adult labels
      - decade stages except the first decade stage (0–9)
      - middle aged stage (>=15)
    Excludes:
      - month-old stages, post-fertilization stages, and NN-year-old < 15
    """
    s = stage.astype("string")

    # 1) numeric year-old stages
    age = pd.to_numeric(s.str.extract(r"^(\d+)-year-old stage$")[0], errors="coerce")
    m_year15 = age.ge(15)

    # 2) explicit adult labels
    m_adult_labels = s.isin(["adult stage", "young adult stage", "prime adult stage", "late adult stage"])

    # 3) decade stages (sixth decade, seventh decade, etc.)
    m_decade = s.str.endswith("decade stage", na=False) & ~s.eq("first decade stage")

    # 4) middle aged stage
    m_middle_aged = s.eq("middle aged stage")

    return (m_year15 | m_adult_labels | m_decade | m_middle_aged)


def main() -> None:
    pf = pq.ParquetFile(INP)

    # Required columns for your filters:
    required = ["assay", "sex", "development_stage"]
    require_columns(pf, required)

    # disease: allow either label or ontology id
    has_disease_label = "disease" in pf.schema_arrow.names
    has_disease_otid  = "disease_ontology_term_id" in pf.schema_arrow.names
    if not (has_disease_label or has_disease_otid):
        raise ValueError("Parquet lacks both 'disease' and 'disease_ontology_term_id' columns.")

    # Discover how adult is stored (optional, but useful)
    adult_pairs = discover_adult_labels_and_ids(pf)
    print("Unique development_stage entries containing 'adult':")
    print(adult_pairs.head(50).to_string(index=False))

    # Columns to read/write (keep only what you need; expand if desired)
    cols = ["assay", "sex", "suspension_type", "development_stage"]
    for c in ["disease", "disease_ontology_term_id",
              "development_stage_ontology_term_id",
              "dataset_id", "donor_id", "tissue_general", "cell_type",
              "is_primary_data"]:
        if c in pf.schema_arrow.names:
            cols.append(c)

    writer: Optional[pq.ParquetWriter] = None
    rows_written = 0
    batches_seen = 0

    try:
        for batch in pf.iter_batches(columns=cols, batch_size=BATCH_SIZE):
            batches_seen += 1
            df = batch.to_pandas()

            # core filters
            m = (
                df["assay"].isin(TARGET_ASSAYS) &
                df["sex"].isin(TARGET_SEXES) &
                adult15_mask(df["development_stage"])
            )

            # disease filter
            if has_disease_otid:
                m = m & (df["disease_ontology_term_id"] == NORMAL_DISEASE_OTID)
            else:
                m = m & (df["disease"].astype("string") == NORMAL_DISEASE_LABEL)

            df2 = df.loc[m].copy()
            if df2.empty:
                continue

            tbl = pa.Table.from_pandas(df2, preserve_index=False)

            if writer is None:
                writer = pq.ParquetWriter(str(OUT), tbl.schema, compression="zstd")
            writer.write_table(tbl)
            rows_written += tbl.num_rows

            if batches_seen % 10 == 0:
                print(f"[progress] batches_seen={batches_seen:,} rows_written={rows_written:,}")

    finally:
        if writer is not None:
            writer.close()

    print(f"Done. Wrote {rows_written:,} rows to:\n  {OUT}")


if __name__ == "__main__":
    main()


Unique development_stage entries containing 'adult':
development_stage development_stage_ontology_term_id
      adult stage                     HsapDv:0000258
 late adult stage                     HsapDv:0000227
prime adult stage                     HsapDv:0000226
young adult stage                     HsapDv:0000266
[progress] batches_seen=10 rows_written=1,480,461
[progress] batches_seen=20 rows_written=3,304,803
[progress] batches_seen=30 rows_written=4,995,514
[progress] batches_seen=40 rows_written=6,193,966
[progress] batches_seen=50 rows_written=7,306,571
[progress] batches_seen=60 rows_written=8,733,869
[progress] batches_seen=70 rows_written=9,551,196
[progress] batches_seen=80 rows_written=10,556,448
[progress] batches_seen=90 rows_written=11,442,345
[progress] batches_seen=110 rows_written=13,670,545
[progress] batches_seen=120 rows_written=15,431,734
[progress] batches_seen=140 rows_written=17,775,774
[progress] batches_seen=150 rows_written=19,872,926
[progress] batches_see

In [17]:
from pathlib import Path
import pyarrow.parquet as pq

p = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")
pf = pq.ParquetFile(p)

print("Number of columns:", len(pf.schema_arrow.names))
print("\nColumn names:")
for c in pf.schema_arrow.names:
    print(" ", c)

print("\nSchema (name: type):")
print(pf.schema_arrow)


Number of columns: 15

Column names:
  dataset_id
  donor_id
  assay
  sex
  tissue
  tissue_general
  cell_type
  disease
  development_stage
  is_primary_data
  tissue_ontology_term_id
  tissue_general_ontology_term_id
  cell_type_ontology_term_id
  disease_ontology_term_id
  development_stage_ontology_term_id

Schema (name: type):
dataset_id: dictionary<values=string, indices=int16, ordered=0>
donor_id: dictionary<values=string, indices=int16, ordered=0>
assay: dictionary<values=string, indices=int8, ordered=0>
sex: dictionary<values=string, indices=int8, ordered=0>
tissue: dictionary<values=string, indices=int16, ordered=0>
tissue_general: dictionary<values=string, indices=int8, ordered=0>
cell_type: dictionary<values=string, indices=int16, ordered=0>
disease: dictionary<values=string, indices=int16, ordered=0>
development_stage: dictionary<values=string, indices=int16, ordered=0>
is_primary_data: bool
tissue_ontology_term_id: dictionary<values=string, indices=int16, ordered=0>
tis

In [18]:
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow as pa

P = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")
pf = pq.ParquetFile(P)

# sanity check
if "dataset_id" not in pf.schema_arrow.names:
    raise ValueError("This parquet does not contain a 'dataset_id' column. Print schema to confirm columns.")

unique_ids = set()
rows_scanned = 0

for batch in pf.iter_batches(columns=["dataset_id"], batch_size=1_000_000):
    rows_scanned += batch.num_rows
    col = batch.column(0)

    # robustly convert to strings (handles dictionary-encoded columns)
    col = pc.cast(col, pa.string())

    # update python set
    unique_ids.update([x for x in col.to_pylist() if x is not None])

print("Rows scanned:", f"{rows_scanned:,}")
print("Unique dataset_id:", len(unique_ids))


Rows scanned: 62,464,283
Unique dataset_id: 469


In [19]:
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow as pa

P = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")
pf = pq.ParquetFile(P)

batch = next(pf.iter_batches(columns=["dataset_id"], batch_size=1000))
col = pc.cast(batch.column(0), pa.string())
vals = [v for v in col.to_pylist() if v is not None]

print("First 25 dataset_id values (may include duplicates):")
print(vals[:25])


First 25 dataset_id values (may include duplicates):
['a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-101e17f263c8', 'a5d95a42-0137-496f-8a60-1

In [20]:
import cellxgene_census

dataset_id = "a5d95a42-0137-496f-8a60-101e17f263c8"

with cellxgene_census.open_soma(census_version="stable") as census:
    ds = (
        census["census_info"]["datasets"]
        .read(value_filter=f"dataset_id == '{dataset_id}'")
        .concat()
        .to_pandas()
    )

ds


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


,soma_joinid,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
0,21,Publication: https://doi.org/10.1038/s41467-02...,bf325905-5e8e-42e3-933d-9a9053e9af80,Single-cell Atlas of common variable immunodef...,10.1038/s41467-022-29450-x,Rodríguez-Ubreva et al. (2022) Nat Commun,a5d95a42-0137-496f-8a60-101e17f263c8,963e004b-77ac-451d-a73c-bd9e1e14c232,Steady-state B cells - scRNA-seq,a5d95a42-0137-496f-8a60-101e17f263c8.h5ad,1324


In [21]:
from pathlib import Path
import pandas as pd
import cellxgene_census as cxc

OUTDIR = Path.home() / "census_metadata_inventory"
OUTDIR.mkdir(parents=True, exist_ok=True)

def schema_to_df(schema):
    return pd.DataFrame({
        "column": schema.names,
        "arrow_type": [str(schema.field(i).type) for i in range(len(schema.names))]
    })

with cxc.open_soma(census_version="stable") as census:
    # List organisms available in this Census build (e.g., homo_sapiens, mus_musculus)
    organisms = list(census["census_data"].keys())

    # List all census_info tables available
    info_tables = list(census["census_info"].keys())
    (OUTDIR / "census_info_tables.txt").write_text("\n".join(info_tables) + "\n")

    # Save schema for each organism’s obs and RNA var
    for org in organisms:
        exp = census["census_data"][org]

        obs_schema = exp.obs.schema
        schema_to_df(obs_schema).to_csv(OUTDIR / f"{org}_obs_schema.csv", index=False)

        var_schema = exp.ms["RNA"].var.schema
        schema_to_df(var_schema).to_csv(OUTDIR / f"{org}_rna_var_schema.csv", index=False)

    # Save schema for key census_info tables (if present)
    for name in ["datasets", "summary_cell_counts"]:
        if name in census["census_info"]:
            df = schema_to_df(census["census_info"][name].schema)
            df.to_csv(OUTDIR / f"census_info_{name}_schema.csv", index=False)

print("Wrote inventory to:", OUTDIR)


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


Wrote inventory to: /home/crizza/census_metadata_inventory


In [1]:
from __future__ import annotations

from collections import Counter
from pathlib import Path
import re

import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq


PARQUET_PATH = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")
OUTDIR = PARQUET_PATH.parent / "dev_stage_inventory"
OUTDIR.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 500_000          # adjust to your RAM (250k–1M is typical)
PROGRESS_EVERY = 20           # print progress every N batches

# Regex for extracting numeric ages from labels like "50-year-old stage"
YEAR_OLD_RE = re.compile(r"^(\d+)-year-old stage$")


def update_counter_from_value_counts(counter: Counter, arr: pa.Array) -> None:
    """Update a Python Counter using Arrow value_counts for a 1D array."""
    arr = pc.cast(arr, pa.string())
    vc = pc.value_counts(arr)  # struct: values, counts
    vals = vc.field("values").to_pylist()
    cts = vc.field("counts").to_pylist()
    for v, c in zip(vals, cts):
        if v is None:
            continue
        counter[v] += int(c)


def main() -> None:
    pf = pq.ParquetFile(PARQUET_PATH)
    cols_present = set(pf.schema_arrow.names)

    required = ["development_stage", "development_stage_ontology_term_id"]
    missing = [c for c in required if c not in cols_present]
    if missing:
        raise ValueError(
            f"Missing required column(s) in Parquet: {missing}\n"
            f"Available columns include: {sorted(list(cols_present))[:50]} ..."
        )

    # Global tallies
    stage_counts = Counter()
    otid_counts = Counter()
    pair_counts = Counter()      # key: (stage, otid) -> count
    year_age_counts = Counter()  # key: int(age_years) -> count
    year_stage_counts = Counter()# key: (age_years, stage, otid) -> count (mostly redundant, but explicit)

    rows_scanned = 0
    batches_seen = 0

    # Iterate through the entire file streaming
    for batch in pf.iter_batches(columns=required, batch_size=BATCH_SIZE):
        batches_seen += 1
        rows_scanned += batch.num_rows

        stage = pc.cast(batch.column(0), pa.string())
        otid  = pc.cast(batch.column(1), pa.string())

        # (1) counts for stage and ontology ids (fast, via Arrow value_counts)
        update_counter_from_value_counts(stage_counts, stage)
        update_counter_from_value_counts(otid_counts, otid)

        # (2) counts for (stage, otid) pairs
        # Arrow can't value_count struct<dict,string> reliably in your build, so do per-batch groupby in pandas.
        df = pa.Table.from_arrays([stage, otid], names=["development_stage", "development_stage_ontology_term_id"]) \
                    .to_pandas()

        gb = df.groupby(["development_stage", "development_stage_ontology_term_id"], dropna=False).size()
        for (s, o), n in gb.items():
            if pd.isna(s) and pd.isna(o):
                continue
            pair_counts[(s, o)] += int(n)

            # (3) extract numeric ages from "N-year-old stage" labels and accumulate
            # Do this on unique pair keys per batch (via gb), not per row.
            if isinstance(s, str):
                m = YEAR_OLD_RE.match(s)
                if m:
                    age = int(m.group(1))
                    year_age_counts[age] += int(n)
                    year_stage_counts[(age, s, o)] += int(n)

        if PROGRESS_EVERY and (batches_seen % PROGRESS_EVERY == 0):
            print(f"[scan] batches={batches_seen:,} rows_scanned={rows_scanned:,} "
                  f"unique_pairs={len(pair_counts):,} unique_stages={len(stage_counts):,}")

    print(f"Done scanning. Rows scanned: {rows_scanned:,}")
    print(f"Unique development_stage values: {len(stage_counts):,}")
    print(f"Unique (stage, otid) pairs: {len(pair_counts):,}")
    print(f"Unique numeric year ages: {len(year_age_counts):,}")

    # ---- Write outputs ----

    # A) full catalog of (stage, otid) pairs with counts
    pairs_df = pd.DataFrame(
        [(s, o, n) for (s, o), n in pair_counts.items()],
        columns=["development_stage", "development_stage_ontology_term_id", "cell_count_in_parquet"]
    ).sort_values("cell_count_in_parquet", ascending=False)
    pairs_csv = OUTDIR / "development_stage_pairs_counts.csv"
    pairs_df.to_csv(pairs_csv, index=False)

    # B) numeric year-old stages with ontology ids + counts (age resolved)
    year_df = pd.DataFrame(
        [(age, s, o, n) for (age, s, o), n in year_stage_counts.items()],
        columns=["age_years", "development_stage", "development_stage_ontology_term_id", "cell_count_in_parquet"]
    ).sort_values(["age_years", "cell_count_in_parquet"], ascending=[True, False])
    year_csv = OUTDIR / "year_old_stage_pairs_counts.csv"
    year_df.to_csv(year_csv, index=False)

    # C) collapsed age summary (counts by numeric age)
    age_df = pd.DataFrame(
        sorted(year_age_counts.items(), key=lambda x: x[0]),
        columns=["age_years", "cell_count_in_parquet"]
    )
    age_csv = OUTDIR / "age_years_counts.csv"
    age_df.to_csv(age_csv, index=False)

    # D) also write a quick “stages only” list (no otid) if helpful
    stage_df = pd.DataFrame(
        stage_counts.most_common(),
        columns=["development_stage", "cell_count_in_parquet"]
    )
    stage_csv = OUTDIR / "development_stage_counts.csv"
    stage_df.to_csv(stage_csv, index=False)

    print("\nWrote outputs to:")
    print(" ", pairs_csv)
    print(" ", year_csv)
    print(" ", age_csv)
    print(" ", stage_csv)


if __name__ == "__main__":
    main()


[scan] batches=20 rows_scanned=7,000,000 unique_pairs=133 unique_stages=133
[scan] batches=40 rows_scanned=13,631,488 unique_pairs=158 unique_stages=158
[scan] batches=60 rows_scanned=20,500,000 unique_pairs=171 unique_stages=171
[scan] batches=80 rows_scanned=27,262,976 unique_pairs=181 unique_stages=181
[scan] batches=100 rows_scanned=34,000,000 unique_pairs=182 unique_stages=182
[scan] batches=120 rows_scanned=40,894,464 unique_pairs=182 unique_stages=182
[scan] batches=140 rows_scanned=47,500,000 unique_pairs=182 unique_stages=182
[scan] batches=160 rows_scanned=54,500,000 unique_pairs=185 unique_stages=185
[scan] batches=180 rows_scanned=61,000,000 unique_pairs=186 unique_stages=186
Done scanning. Rows scanned: 62,464,283
Unique development_stage values: 186
Unique (stage, otid) pairs: 186
Unique numeric year ages: 98

Wrote outputs to:
  /home/crizza/census_exports/dev_stage_inventory/development_stage_pairs_counts.csv
  /home/crizza/census_exports/dev_stage_inventory/year_old_st

In [22]:
import cellxgene_census as cxc

with cxc.open_soma(census_version="stable") as census:
    ds = census["census_info"]["datasets"].read().concat().to_pandas()

print(ds.columns.tolist())
print(ds.head(3).T)


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


['soma_joinid', 'citation', 'collection_id', 'collection_name', 'collection_doi', 'collection_doi_label', 'dataset_id', 'dataset_version_id', 'dataset_title', 'dataset_h5ad_path', 'dataset_total_cell_count']
                                                                          0  \
soma_joinid                                                               0   
citation                  Publication: https://doi.org/10.1016/j.isci.20...   
collection_id                          8e880741-bf9a-4c8e-9227-934204631d2a   
collection_name           High Resolution Slide-seqV2 Spatial Transcript...   
collection_doi                                   10.1016/j.isci.2022.104097   
collection_doi_label                        Marshall et al. (2022) iScience   
dataset_id                             4eb29386-de81-452f-b3c0-e00844e8c7fd   
dataset_version_id                     66699060-0389-4fbd-b3a5-196b3b4e32d6   
dataset_title              Spatial transcriptomics in mouse: Puck_191112_05   
da

In [3]:
from pathlib import Path
import pandas as pd

INVENTORY_DIR = Path("/home/crizza/census_exports/dev_stage_inventory")

year_pairs = pd.read_csv(INVENTORY_DIR / "year_old_stage_pairs_counts.csv")
pairs      = pd.read_csv(INVENTORY_DIR / "development_stage_pairs_counts.csv")


# 1) All exact "N-year-old stage" where N >= 15
ids_ge15 = set(
    year_pairs.loc[year_pairs["age_years"] >= 15, "development_stage_ontology_term_id"]
)

# 2) Add non-1-year-bin adult-ish buckets/ranges that show up in Census exports
extra_labels = [
    "15-19 year-old",
    "adult stage",
    "young adult stage",
    "prime adult stage",
    "late adult stage",
    "middle aged stage",
    "third decade stage",
    "fourth decade stage",
    "fifth decade stage",
    "sixth decade stage",
    "seventh decade stage",
    "eighth decade stage",
    "ninth decade stage",
    "60-79 year-old stage",
    "80 year-old and over stage",
    "90 year-old and over stage",
]
ids_ge15 |= set(
    pairs.loc[pairs["development_stage"].isin(extra_labels), "development_stage_ontology_term_id"]
)

# 3) Clean-up
ids_ge15.discard("unknown")
ids_ge15 = sorted(ids_ge15)

print("Number of development_stage term IDs for age >= 15:", len(ids_ge15))
print("Example IDs:", ids_ge15[:10])


Number of development_stage term IDs for age >= 15: 100
Example IDs: ['HsapDv:0000095', 'HsapDv:0000109', 'HsapDv:0000110', 'HsapDv:0000111', 'HsapDv:0000112', 'HsapDv:0000113', 'HsapDv:0000114', 'HsapDv:0000115', 'HsapDv:0000116', 'HsapDv:0000117']


In [4]:
from __future__ import annotations

from pathlib import Path
from typing import List, Dict

import pandas as pd
import cellxgene_census as cxc


# -----------------------------
# User parameters
# -----------------------------
ORGANISM = "homo_sapiens"

NORMAL_DISEASE_OTID = "PATO:0000461"  # normal
TARGET_ASSAY_LABELS = ["10x 3' v2", "10x 3' v3", "10x multiome"]
TARGET_SEXES = ["male", "female"]
TARGET_SUSPENSION_TYPE = "cell"

MIN_AGE_YEARS = 15

# Point this to where you wrote the inventory CSVs
INVENTORY_DIR = Path("/home/crizza/census_exports/dev_stage_inventory")
# (If you are using the uploaded files in this chat, set INVENTORY_DIR = Path("/mnt/data"))


# -----------------------------
# Helpers
# -----------------------------
def _format_in_list(values: List[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def build_stage_ids_ge15(inventory_dir: Path, min_age_years: int = 15) -> List[str]:
    """
    Construct a development_stage_ontology_term_id allow-list for age >= min_age_years
    using your inventory CSVs:
      - year_old_stage_pairs_counts.csv
      - development_stage_pairs_counts.csv
    """
    year_pairs = pd.read_csv(inventory_dir / "year_old_stage_pairs_counts.csv")
    pairs = pd.read_csv(inventory_dir / "development_stage_pairs_counts.csv")

    # 1) All exact N-year-old stages where N >= 15
    ids = set(
        year_pairs.loc[year_pairs["age_years"] >= min_age_years, "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )

    # 2) Add adult/older bins and labels (these appear in your file)
    extra_labels = [
        "15-19 year-old",
        "adult stage",
        "young adult stage",
        "prime adult stage",
        "late adult stage",
        "middle aged stage",
        "third decade stage",
        "fourth decade stage",
        "fifth decade stage",
        "sixth decade stage",
        "seventh decade stage",
        "eighth decade stage",
        "ninth decade stage",
        "60-79 year-old stage",
        "80 year-old and over stage",
        "90 year-old and over stage",
    ]

    extra_ids = (
        pairs.loc[pairs["development_stage"].isin(extra_labels), "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )
    ids.update(extra_ids)

    # 3) Remove unknown (if present)
    ids.discard("unknown")

    return sorted(ids)


def lookup_assay_term_ids(census) -> Dict[str, str]:
    """
    Map assay label -> assay_ontology_term_id using census_info/summary_cell_counts,
    avoiding quoting issues with "10x 3' v3".
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )
    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(TARGET_ASSAY_LABELS))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in TARGET_ASSAY_LABELS if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for assay label(s): {missing}")
    return found


def build_obs_value_filter(census, inventory_dir: Path) -> str:
    """
    Build a single combined obs value_filter implementing:
      - normal
      - male/female
      - suspension_type == cell
      - assay is 10x v2/v3/multiome (via ontology IDs)
      - development_stage >= 15 years (via allow-list of stage ontology IDs)
    """
    # Resolve assay ontology IDs
    assay_map = lookup_assay_term_ids(census)
    assay_term_ids = [assay_map[x] for x in TARGET_ASSAY_LABELS]

    # Resolve developmental-stage ontology IDs for >=15 years
    stage_ids_ge15 = build_stage_ids_ge15(inventory_dir, MIN_AGE_YEARS)

    vf_parts = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{NORMAL_DISEASE_OTID}'",
        f"sex in {_format_in_list(TARGET_SEXES)}",
        f"suspension_type == '{TARGET_SUSPENSION_TYPE}'",
        f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}",
        f"development_stage_ontology_term_id in {_format_in_list(stage_ids_ge15)}",
    ]
    return " and ".join(vf_parts)


# -----------------------------
# Example: produce filter and use it
# -----------------------------
with cxc.open_soma(census_version="stable") as census:
    human = census["census_data"][ORGANISM]

    value_filter = build_obs_value_filter(census, INVENTORY_DIR)
    print("Combined OBS value_filter:\n", value_filter)

    # Example read (cell metadata only)
    # tbl_iter = human.obs.read(value_filter=value_filter, column_names=[...])


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


Combined OBS value_filter:
 is_primary_data == True and disease_ontology_term_id == 'PATO:0000461' and sex in ['male', 'female'] and suspension_type == 'cell' and assay_ontology_term_id in ['EFO:0009899', 'EFO:0009922', 'EFO:0030059'] and development_stage_ontology_term_id in ['HsapDv:0000095', 'HsapDv:0000109', 'HsapDv:0000110', 'HsapDv:0000111', 'HsapDv:0000112', 'HsapDv:0000113', 'HsapDv:0000114', 'HsapDv:0000115', 'HsapDv:0000116', 'HsapDv:0000117', 'HsapDv:0000118', 'HsapDv:0000119', 'HsapDv:0000120', 'HsapDv:0000121', 'HsapDv:0000122', 'HsapDv:0000123', 'HsapDv:0000124', 'HsapDv:0000125', 'HsapDv:0000126', 'HsapDv:0000127', 'HsapDv:0000128', 'HsapDv:0000129', 'HsapDv:0000130', 'HsapDv:0000131', 'HsapDv:0000132', 'HsapDv:0000133', 'HsapDv:0000134', 'HsapDv:0000135', 'HsapDv:0000136', 'HsapDv:0000137', 'HsapDv:0000138', 'HsapDv:0000139', 'HsapDv:0000140', 'HsapDv:0000141', 'HsapDv:0000142', 'HsapDv:0000143', 'HsapDv:0000144', 'HsapDv:0000145', 'HsapDv:0000146', 'HsapDv:0000147', 'H

In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Optional

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import cellxgene_census as cxc

### Current Version 
# -----------------------------
# Configuration
# -----------------------------
CENSUS_VERSION = "stable"
ORGANISM = "homo_sapiens"

# Filters requested
NORMAL_DISEASE_OTID = "PATO:0000461"  # normal
TARGET_ASSAY_LABELS = ["10x 3' v2", "10x 3' v3", "10x multiome"]
TARGET_SEXES = ["male", "female"]
TARGET_SUSPENSION_TYPE = "cell"
MIN_AGE_YEARS = 15

# Inventory CSVs directory (set this explicitly if you know it)
# Example: Path("/home/crizza/census_exports/dev_stage_inventory")
INVENTORY_DIR: Optional[Path] = None

# Output
OUTDIR = Path(os.environ.get("CENSUS_OUTDIR", str(Path.home() / "census_exports")))
OUTFILE = "census_hsapiens_normal_cell_mf_10x_age15plus_obs.parquet"
OVERWRITE = True

# Columns to export (keep compact; anything missing will be skipped safely)
OBS_COLS = [
    "dataset_id",
    "donor_id",
    "assay",
    "assay_ontology_term_id",
    "sex",
    "suspension_type",
    "disease",
    "disease_ontology_term_id",
    "development_stage",
    "development_stage_ontology_term_id",
    "tissue",
    "tissue_general",
    "cell_type",
    "is_primary_data",
]


# -----------------------------
# Helpers
# -----------------------------
def ensure_writable_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path}")


def resolve_outpath(outdir: Path, outfile: str, overwrite: bool) -> Path:
    ensure_writable_dir(outdir)
    outpath = outdir / outfile
    if outpath.exists():
        if overwrite:
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output exists and OVERWRITE=False: {outpath}")
    return outpath


def _format_in_list(values: List[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def detect_inventory_dir(user_hint: Optional[Path] = None) -> Path:
    """
    Locate the folder containing:
      - year_old_stage_pairs_counts.csv
      - development_stage_pairs_counts.csv
    """
    required = ["year_old_stage_pairs_counts.csv", "development_stage_pairs_counts.csv"]

    candidates = []
    if user_hint is not None:
        candidates.append(user_hint)

    # common locations
    candidates.extend([
        Path("/home/crizza/census_exports/dev_stage_inventory"),
        Path.home() / "census_exports" / "dev_stage_inventory",
        Path("/mnt/data"),  # in-chat uploads
    ])

    for d in candidates:
        if all((d / f).exists() for f in required):
            return d

    raise FileNotFoundError(
        "Could not find inventory CSVs. Set INVENTORY_DIR to the folder containing:\n"
        "  - year_old_stage_pairs_counts.csv\n"
        "  - development_stage_pairs_counts.csv"
    )


def build_stage_ids_ge15(inventory_dir: Path, min_age_years: int = 15) -> List[str]:
    """
    Build development_stage_ontology_term_id allow-list for age >= min_age_years using your CSV inventories.
    - Includes all N-year-old stages where N >= min_age_years
    - Adds adult bins/labels seen in your data (adult stage, decades, 15-19, 80+ etc.)
    """
    year_pairs = pd.read_csv(inventory_dir / "year_old_stage_pairs_counts.csv")
    pairs = pd.read_csv(inventory_dir / "development_stage_pairs_counts.csv")

    ids = set(
        year_pairs.loc[year_pairs["age_years"] >= min_age_years, "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )

    extra_labels = [
        "15-19 year-old",
        "adult stage",
        "young adult stage",
        "prime adult stage",
        "late adult stage",
        "middle aged stage",
        "third decade stage",
        "fourth decade stage",
        "fifth decade stage",
        "sixth decade stage",
        "seventh decade stage",
        "eighth decade stage",
        "ninth decade stage",
        "60-79 year-old stage",
        "80 year-old and over stage",
        "90 year-old and over stage",
    ]

    extra_ids = (
        pairs.loc[pairs["development_stage"].isin(extra_labels), "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )
    ids.update(extra_ids)

    ids.discard("unknown")
    return sorted(ids)


def lookup_assay_term_ids(census) -> Dict[str, str]:
    """
    Map assay label -> assay_ontology_term_id via census_info/summary_cell_counts.
    This avoids issues with apostrophes in assay labels.
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )
    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(TARGET_ASSAY_LABELS))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in TARGET_ASSAY_LABELS if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for assay label(s): {missing}")
    return found


def pick_existing_cols(schema_names: List[str], desired: List[str]) -> List[str]:
    s = set(schema_names)
    return [c for c in desired if c in s]


def build_obs_value_filter(
    census,
    inventory_dir: Path,
) -> str:
    assay_map = lookup_assay_term_ids(census)
    assay_term_ids = [assay_map[x] for x in TARGET_ASSAY_LABELS]
    stage_ids_ge15 = build_stage_ids_ge15(inventory_dir, MIN_AGE_YEARS)

    vf_parts = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{NORMAL_DISEASE_OTID}'",
        f"sex in {_format_in_list(TARGET_SEXES)}",
        f"suspension_type == '{TARGET_SUSPENSION_TYPE}'",
        f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}",
        f"development_stage_ontology_term_id in {_format_in_list(stage_ids_ge15)}",
    ]
    return " and ".join(vf_parts)


def export_obs_to_parquet(
    exp,
    value_filter: str,
    outpath: Path,
    obs_cols: Optional[List[str]] = None,
) -> None:
    """
    Stream obs rows matching value_filter and write to a single parquet.
    No BatchSize required; TileDB-SOMA will still yield an iterator of Arrow tables.
    """
    schema_names = exp.obs.schema.names
    cols = pick_existing_cols(schema_names, obs_cols) if obs_cols else None

    it = exp.obs.read(value_filter=value_filter, column_names=cols)

    writer: Optional[pq.ParquetWriter] = None
    rows = 0
    batches = 0

    try:
        for tbl in it:
            if not isinstance(tbl, pa.Table):
                tbl = pa.Table.from_batches([tbl])
            if tbl.num_rows == 0:
                continue

            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema, compression="zstd")

            writer.write_table(tbl)
            rows += tbl.num_rows
            batches += 1

            if batches % 10 == 0:
                print(f"[export] batches={batches:,} rows={rows:,}")

    finally:
        if writer is not None:
            writer.close()

    print(f"Done. Wrote {rows:,} rows across {batches:,} batches to:\n  {outpath}")


# -----------------------------
# Main
# -----------------------------
def main() -> None:
    inventory_dir = detect_inventory_dir(INVENTORY_DIR)
    print("Using inventory CSV directory:", inventory_dir)

    outpath = resolve_outpath(OUTDIR, OUTFILE, OVERWRITE)
    print("Output file:", outpath)

    with cxc.open_soma(census_version=CENSUS_VERSION) as census:
        exp = census["census_data"][ORGANISM]

        value_filter = build_obs_value_filter(census, inventory_dir)
        print("\nCombined OBS value_filter:\n", value_filter, "\n")

        export_obs_to_parquet(exp, value_filter, outpath, obs_cols=OBS_COLS)


if __name__ == "__main__":
    main()


Using inventory CSV directory: /home/crizza/census_exports/dev_stage_inventory
Output file: /home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_age15plus_obs.parquet


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.



Combined OBS value_filter:
 is_primary_data == True and disease_ontology_term_id == 'PATO:0000461' and sex in ['male', 'female'] and suspension_type == 'cell' and assay_ontology_term_id in ['EFO:0009899', 'EFO:0009922', 'EFO:0030059'] and development_stage_ontology_term_id in ['HsapDv:0000095', 'HsapDv:0000109', 'HsapDv:0000110', 'HsapDv:0000111', 'HsapDv:0000112', 'HsapDv:0000113', 'HsapDv:0000114', 'HsapDv:0000115', 'HsapDv:0000116', 'HsapDv:0000117', 'HsapDv:0000118', 'HsapDv:0000119', 'HsapDv:0000120', 'HsapDv:0000121', 'HsapDv:0000122', 'HsapDv:0000123', 'HsapDv:0000124', 'HsapDv:0000125', 'HsapDv:0000126', 'HsapDv:0000127', 'HsapDv:0000128', 'HsapDv:0000129', 'HsapDv:0000130', 'HsapDv:0000131', 'HsapDv:0000132', 'HsapDv:0000133', 'HsapDv:0000134', 'HsapDv:0000135', 'HsapDv:0000136', 'HsapDv:0000137', 'HsapDv:0000138', 'HsapDv:0000139', 'HsapDv:0000140', 'HsapDv:0000141', 'HsapDv:0000142', 'HsapDv:0000143', 'HsapDv:0000144', 'HsapDv:0000145', 'HsapDv:0000146', 'HsapDv:0000147', '

In [6]:
from pathlib import Path
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq

P = Path("/home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_age15plus_obs.parquet")
pf = pq.ParquetFile(P)

if "dataset_id" not in pf.schema_arrow.names:
    raise ValueError("Column 'dataset_id' not found. Print pf.schema_arrow.names to confirm.")

unique_ids = set()
rows_scanned = 0

for batch in pf.iter_batches(columns=["dataset_id"], batch_size=1_000_000):
    rows_scanned += batch.num_rows
    col = pc.cast(batch.column(0), pa.string())

    vc = pc.value_counts(col)  # struct array with fields: values, counts
    vals = vc.field("values").to_pylist()
    unique_ids.update([v for v in vals if v is not None])

print("Rows scanned:", f"{rows_scanned:,}")
print("Unique dataset_id:", len(unique_ids))

# show examples (optional)
examples = sorted(unique_ids)[:20]
print("Example dataset_id values:")
for x in examples:
    print(" ", x)


Rows scanned: 20,327,572
Unique dataset_id: 165
Example dataset_id values:
  0087cde2-967d-4f7c-8e6e-40e4c9ad1891
  01209dce-3575-4bed-b1df-129f57fbc031
  019c7af2-c827-4454-9970-44d5e39ce068
  0436a180-cb44-47ba-8ffa-807b7a468469
  093d3bfe-6f0f-4ac0-a7a1-829f94d0a49f
  0b4a15a7-4e9e-4555-9733-2423e5c66469
  0ba636a1-4754-4786-a8be-7ab3cf760fd6
  0bae7ebf-eb54-46a6-be9a-3461cecefa4c
  0c9a8cfb-6649-4d52-b418-6d8e56bd7afe
  0de831e0-a525-4ec5-b717-df56f2de2bf0
  0f4865d5-8000-4f68-8ac7-f5efea9e5e70
  124744b8-4681-474a-9894-683896122708
  12c3efd8-d515-42e6-b93c-4961e599d524
  14363b6e-3428-45b0-8704-10fd5d6316df
  149b2c3f-ee11-47a7-984b-923570280bd7
  15d374d6-0dfd-4d8e-ade7-81f73dc921ee
  16023185-de21-4c0d-a9c8-73abdd52d142
  171b03c9-516b-499c-a434-fa4f70191988
  19053a82-9c89-4fb8-bd19-d7b1800b0b7b
  19e46756-9100-4e01-8b0e-23b557558a4c


In [7]:
from pathlib import Path
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq
import pandas as pd
import cellxgene_census as cxc

PARQ = Path("/home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_age15plus_obs.parquet")
OUT  = Path("/home/crizza/census_exports/dataset_metadata_for_filtered165.csv")

# 1) Collect unique dataset_ids from parquet
pf = pq.ParquetFile(PARQ)
unique_ids = set()

for batch in pf.iter_batches(columns=["dataset_id"], batch_size=1_000_000):
    col = pc.cast(batch.column(0), pa.string())
    vc = pc.value_counts(col)
    vals = vc.field("values").to_pylist()
    unique_ids.update([v for v in vals if v is not None])

ids = sorted(unique_ids)
print("Unique dataset_id:", len(ids))

# 2) Fetch dataset metadata from census_info["datasets"]
def chunks(lst, n=250):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

rows = []
with cxc.open_soma(census_version="stable") as census:
    ds_tbl = census["census_info"]["datasets"]

    for ch in chunks(ids, 250):
        clause = " or ".join([f"dataset_id == '{x}'" for x in ch])
        df = ds_tbl.read(value_filter=clause).concat().to_pandas()
        rows.append(df)

meta = pd.concat(rows, ignore_index=True).drop_duplicates(subset=["dataset_id"])

# Keep useful columns if present
keep = [c for c in [
    "dataset_id",
    "dataset_title",
    "dataset_total_cell_count",
    "collection_id",
    "collection_name",
    "collection_doi",
    "collection_doi_label",
    "dataset_version_id",
    "citation",
] if c in meta.columns]

meta = meta[keep].sort_values(["collection_name", "dataset_title"])
meta.to_csv(OUT, index=False)

print("Wrote:", OUT)
print("Rows:", meta.shape[0])


Unique dataset_id: 165


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.


Wrote: /home/crizza/census_exports/dataset_metadata_for_filtered165.csv
Rows: 165


In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Optional

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import cellxgene_census as cxc

### Current Version 
# -----------------------------
# Configuration
# -----------------------------
CENSUS_VERSION = "stable"
ORGANISM = "homo_sapiens"

# Filters requested
NORMAL_DISEASE_OTID = "PATO:0000461"  # normal
TARGET_ASSAY_LABELS = ["10x 3' v2", "10x 3' v3", "10x multiome"]
TARGET_SEXES = ["male", "female"]
TARGET_SUSPENSION_TYPE = "cell"
MIN_AGE_YEARS = 15

# Inventory CSVs directory (set this explicitly if you know it)
# Example: Path("/home/crizza/census_exports/dev_stage_inventory")
INVENTORY_DIR: Optional[Path] = None

# Output
OUTDIR = Path(os.environ.get("CENSUS_OUTDIR", str(Path.home() / "census_exports")))
OUTFILE = "census_hsapiens_normal_cell_mf_10x_age15plus_obs.parquet"
OVERWRITE = True

# Columns to export (keep compact; anything missing will be skipped safely)
OBS_COLS = [
    "dataset_id",
    "donor_id",
    "assay",
    "assay_ontology_term_id",
    "sex",
    "suspension_type",
    "disease",
    "disease_ontology_term_id",
    "development_stage",
    "development_stage_ontology_term_id",
    "tissue",
    "tissue_general",
    "cell_type",
    "is_primary_data",
]

# -----------------------------
# Helpers
# -----------------------------
def list_obs_columns(census, organism: str) -> List[str]:
    """Return all available obs column names for this organism in the current Census version."""
    exp = census["census_data"][organism]
    return list(exp.obs.schema.names)


def build_obs_columns_no_ontology(
    all_obs_cols: List[str],
    require: Optional[List[str]] = None,
    drop_soma_internal: bool = True,
) -> List[str]:
    """
    Keep all obs columns except ontology term IDs.
    Force-include any 'require' fields if present.
    """
    require = require or []

    # Drop ontology ID columns
    cols = [c for c in all_obs_cols if "ontology_term_id" not in c]

    # Optionally drop internal SOMA join index
    if drop_soma_internal:
        cols = [c for c in cols if c != "soma_joinid"]

    # Ensure required fields exist
    missing = [c for c in require if c not in all_obs_cols]
    if missing:
        # Provide a helpful hint for near-matches
        hints = []
        for m in missing:
            near = [c for c in all_obs_cols if m.split("_")[0] in c]
            hints.append((m, near[:20]))
        msg = "Required obs columns not found in this Census build:\n"
        msg += "\n".join([f"  - {m} (near matches: {near})" for m, near in hints])
        raise ValueError(msg)

    # Force-include required fields (and preserve order)
    for r in require:
        if r not in cols:
            cols.append(r)

    return cols


# -----------------------------
# Main
# -----------------------------
def main(value_filter: str, out_h5ad: Path) -> None:
    with cxc.open_soma(census_version=CENSUS_VERSION) as census:
        all_obs_cols = list_obs_columns(census, ORGANISM)
        obs_cols = build_obs_columns_no_ontology(
            all_obs_cols,
            require=REQUIRE_OBS_FIELDS,
            drop_soma_internal=True,
        )

        # Optional: choose a minimal var schema; or omit var_column_names to take defaults.
        # If you want *all* genes, omit var_column_names; by default Census fetches all genes. :contentReference[oaicite:2]{index=2}
        var_cols = ["feature_id", "feature_name"]

        adata = cxc.get_anndata(
            census,
            organism=ORGANISM,
            obs_value_filter=value_filter,
            obs_column_names=obs_cols,
            var_column_names=var_cols,
        )  # :contentReference[oaicite:3]{index=3}

    # Defensive: ensure no ontology ID columns slipped in
    drop_cols = [c for c in adata.obs.columns if "ontology_term_id" in c]
    if drop_cols:
        adata.obs.drop(columns=drop_cols, inplace=True)

    # Write
    out_h5ad.parent.mkdir(parents=True, exist_ok=True)
    adata.write_h5ad(out_h5ad)
    print(f"Wrote: {out_h5ad}  (cells={adata.n_obs:,}, genes={adata.n_vars:,})")


In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Optional

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import cellxgene_census as cxc


# -----------------------------
# Configuration
# -----------------------------
CENSUS_VERSION = "stable"
ORGANISM = "homo_sapiens"

# Filters requested
NORMAL_DISEASE_OTID = "PATO:0000461"  # normal
TARGET_ASSAY_LABELS = ["10x 3' v2", "10x 3' v3", "10x multiome"]
TARGET_SEXES = ["male", "female"]
TARGET_SUSPENSION_TYPE = "cell"
MIN_AGE_YEARS = 15

# Inventory CSVs directory (leave None to auto-detect)
INVENTORY_DIR: Optional[Path] = None

# Output
OUTDIR = Path("/home/crizza/census_exports")
OUTFILE = "census_hsapiens_normal_cell_mf_10x_age15plus_obs_donor_ethnicity.parquet"
OVERWRITE = True

# Required metadata columns to include in the exported parquet
REQUIRED_EXPORT_COLS = ["dataset_id", "donor_id", "self_reported_ethnicity", ""]


# -----------------------------
# Helpers
# -----------------------------
def ensure_writable_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path}")


def resolve_outpath(outdir: Path, outfile: str, overwrite: bool) -> Path:
    ensure_writable_dir(outdir)
    outpath = outdir / outfile
    if outpath.exists():
        if overwrite:
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output exists and OVERWRITE=False: {outpath}")
    return outpath


def _format_in_list(values: List[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def detect_inventory_dir(user_hint: Optional[Path] = None) -> Path:
    """
    Locate the folder containing:
      - year_old_stage_pairs_counts.csv
      - development_stage_pairs_counts.csv
    """
    required = ["year_old_stage_pairs_counts.csv", "development_stage_pairs_counts.csv"]
    candidates: List[Path] = []
    if user_hint is not None:
        candidates.append(user_hint)

    candidates.extend([
        Path("/home/crizza/census_exports/dev_stage_inventory"),
        Path.home() / "census_exports" / "dev_stage_inventory",
        Path("/mnt/data"),  # if running inside this chat session
    ])

    for d in candidates:
        if all((d / f).exists() for f in required):
            return d

    raise FileNotFoundError(
        "Could not find inventory CSVs. Set INVENTORY_DIR to the folder containing:\n"
        "  - year_old_stage_pairs_counts.csv\n"
        "  - development_stage_pairs_counts.csv"
    )


def build_stage_ids_ge15(inventory_dir: Path, min_age_years: int = 15) -> List[str]:
    """
    Build development_stage_ontology_term_id allow-list for age >= min_age_years using your CSV inventories.
    """
    year_pairs = pd.read_csv(inventory_dir / "year_old_stage_pairs_counts.csv")
    pairs = pd.read_csv(inventory_dir / "development_stage_pairs_counts.csv")

    # 1) All exact N-year-old stages where N >= min_age_years
    ids = set(
        year_pairs.loc[year_pairs["age_years"] >= min_age_years, "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )

    # 2) Add adult bins/labels seen in your data
    extra_labels = [
        "15-19 year-old",
        "adult stage",
        "young adult stage",
        "prime adult stage",
        "late adult stage",
        "middle aged stage",
        "third decade stage",
        "fourth decade stage",
        "fifth decade stage",
        "sixth decade stage",
        "seventh decade stage",
        "eighth decade stage",
        "ninth decade stage",
        "60-79 year-old stage",
        "80 year-old and over stage",
        "90 year-old and over stage",
    ]

    extra_ids = (
        pairs.loc[pairs["development_stage"].isin(extra_labels), "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )
    ids.update(extra_ids)

    ids.discard("unknown")
    return sorted(ids)


def lookup_assay_term_ids(census) -> Dict[str, str]:
    """
    Map assay label -> assay_ontology_term_id via census_info/summary_cell_counts.
    Avoids quoting issues with "10x 3' v3".
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )
    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(TARGET_ASSAY_LABELS))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in TARGET_ASSAY_LABELS if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for assay label(s): {missing}")
    return found


def build_obs_value_filter(census, inventory_dir: Path) -> str:
    """
    Combined filters:
      - is_primary_data
      - normal disease
      - male/female
      - suspension_type cell
      - assays: 10x v2/v3/multiome (via assay_ontology_term_id)
      - age >= 15 years (via development_stage_ontology_term_id allow-list)
    """
    assay_map = lookup_assay_term_ids(census)
    assay_term_ids = [assay_map[x] for x in TARGET_ASSAY_LABELS]
    stage_ids_ge15 = build_stage_ids_ge15(inventory_dir, MIN_AGE_YEARS)

    vf_parts = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{NORMAL_DISEASE_OTID}'",
        f"sex in {_format_in_list(TARGET_SEXES)}",
        f"suspension_type == '{TARGET_SUSPENSION_TYPE}'",
        f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}",
        f"development_stage_ontology_term_id in {_format_in_list(stage_ids_ge15)}",
    ]
    return " and ".join(vf_parts)


def build_export_columns(exp_obs_schema_names: List[str]) -> List[str]:
    """
    Export ALL non-ontology columns + required columns.
    Drops any column containing 'ontology_term_id' from OUTPUT.
    """
    all_cols = list(exp_obs_schema_names)

    # Validate required columns exist
    missing = [c for c in REQUIRED_EXPORT_COLS if c not in all_cols]
    if missing:
        # Suggest likely alternatives (especially for ethnicity)
        hints = {}
        for m in missing:
            key = m.split("_")[0]
            hints[m] = [c for c in all_cols if key in c.lower()][:50]

        msg = "Required export column(s) not found in exp.obs schema:\n"
        for m in missing:
            msg += f"  - {m}\n"
            if hints.get(m):
                msg += f"    suggestions: {hints[m]}\n"
        raise ValueError(msg)

    # Start from all columns, drop ontology IDs + internal join id
    cols = [c for c in all_cols if ("ontology_term_id" not in c and c != "soma_joinid")]

    # Ensure required cols included (in case they were filtered out for any reason)
    for r in REQUIRED_EXPORT_COLS:
        if r not in cols:
            cols.append(r)

    return cols


def export_obs_to_parquet(exp, value_filter: str, outpath: Path, column_names: List[str]) -> None:
    """
    Stream obs rows and write to a single parquet.
    Avoids tiledbsoma BatchSize import entirely.
    """
    it = exp.obs.read(value_filter=value_filter, column_names=column_names)

    writer: Optional[pq.ParquetWriter] = None
    rows = 0
    batches = 0

    try:
        for tbl in it:
            if isinstance(tbl, pa.RecordBatch):
                tbl = pa.Table.from_batches([tbl])
            if not isinstance(tbl, pa.Table):
                tbl = pa.Table.from_batches(tbl.to_batches())

            if tbl.num_rows == 0:
                continue

            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema, compression="zstd")
            writer.write_table(tbl)

            rows += tbl.num_rows
            batches += 1
            if batches % 10 == 0:
                print(f"[export] batches={batches:,} rows={rows:,}")

    finally:
        if writer is not None:
            writer.close()

    print(f"Done. Wrote {rows:,} rows across {batches:,} batches to:\n  {outpath}")


# -----------------------------
# Main
# -----------------------------
def main() -> None:
    inventory_dir = detect_inventory_dir(INVENTORY_DIR)
    print("Using inventory CSV directory:", inventory_dir)

    outpath = resolve_outpath(OUTDIR, OUTFILE, OVERWRITE)
    print("Output file:", outpath)

    with cxc.open_soma(census_version=CENSUS_VERSION) as census:
        exp = census["census_data"][ORGANISM]

        value_filter = build_obs_value_filter(census, inventory_dir)
        print("\nCombined OBS value_filter:\n", value_filter, "\n")

        export_cols = build_export_columns(exp.obs.schema.names)
        print(f"Exporting {len(export_cols)} obs columns (ontology_term_id columns dropped from output).")

        export_obs_to_parquet(exp, value_filter, outpath, export_cols)


if __name__ == "__main__":
    main()


Using inventory CSV directory: /home/crizza/census_exports/dev_stage_inventory
Output file: /home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_age15plus_obs_donor_ethnicity.parquet


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.



Combined OBS value_filter:
 is_primary_data == True and disease_ontology_term_id == 'PATO:0000461' and sex in ['male', 'female'] and suspension_type == 'cell' and assay_ontology_term_id in ['EFO:0009899', 'EFO:0009922', 'EFO:0030059'] and development_stage_ontology_term_id in ['HsapDv:0000095', 'HsapDv:0000109', 'HsapDv:0000110', 'HsapDv:0000111', 'HsapDv:0000112', 'HsapDv:0000113', 'HsapDv:0000114', 'HsapDv:0000115', 'HsapDv:0000116', 'HsapDv:0000117', 'HsapDv:0000118', 'HsapDv:0000119', 'HsapDv:0000120', 'HsapDv:0000121', 'HsapDv:0000122', 'HsapDv:0000123', 'HsapDv:0000124', 'HsapDv:0000125', 'HsapDv:0000126', 'HsapDv:0000127', 'HsapDv:0000128', 'HsapDv:0000129', 'HsapDv:0000130', 'HsapDv:0000131', 'HsapDv:0000132', 'HsapDv:0000133', 'HsapDv:0000134', 'HsapDv:0000135', 'HsapDv:0000136', 'HsapDv:0000137', 'HsapDv:0000138', 'HsapDv:0000139', 'HsapDv:0000140', 'HsapDv:0000141', 'HsapDv:0000142', 'HsapDv:0000143', 'HsapDv:0000144', 'HsapDv:0000145', 'HsapDv:0000146', 'HsapDv:0000147', '

In [9]:
from __future__ import annotations

from collections import Counter, defaultdict
from pathlib import Path

import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq


# -----------------------------
# Config
# -----------------------------
PARQUET_PATH = Path("/home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_age15plus_obs_donor_ethnicity.parquet")
OUTDIR = PARQUET_PATH.parent / "metadata_inventory"   # analogous to dev_stage_inventory
OUTDIR.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 1_000_000
PROGRESS_EVERY = 10

COL_DONOR = "donor_id"
COL_SEX = "sex"
COL_ETH = "self_reported_ethnicity"


# -----------------------------
# Helpers
# -----------------------------
def _require_cols(pf: pq.ParquetFile, cols: list[str]) -> None:
    missing = [c for c in cols if c not in pf.schema_arrow.names]
    if missing:
        raise ValueError(
            "Missing required columns in parquet: "
            + ", ".join(missing)
            + "\nAvailable columns include: "
            + ", ".join(pf.schema_arrow.names[:80])
            + (" ..." if len(pf.schema_arrow.names) > 80 else "")
        )


def _norm_str(x) -> str:
    """Normalize string-like metadata consistently for grouping."""
    if x is None:
        return "unknown"
    s = str(x).strip()
    return s if s else "unknown"


# -----------------------------
# Main scan
# -----------------------------
def main() -> None:
    pf = pq.ParquetFile(PARQUET_PATH)
    _require_cols(pf, [COL_DONOR, COL_SEX, COL_ETH])

    # Cell-level counts (like your development_stage_counts)
    cell_by_sex = Counter()
    cell_by_eth = Counter()
    cell_by_sex_eth = Counter()

    # Donor-level aggregation
    donors_all = set()
    donors_by_sex = defaultdict(set)        # sex -> set(donor_id)
    donors_by_eth = defaultdict(set)        # eth -> set(donor_id)
    donors_by_sex_eth = defaultdict(set)    # (sex, eth) -> set(donor_id)

    donor_cell_count = Counter()            # donor_id -> total cells
    donor_sex = {}                          # donor_id -> canonical sex
    donor_eth = {}                          # donor_id -> canonical ethnicity
    donor_inconsistency = Counter()         # donor_id -> count of conflicting assignments (sex/eth)

    rows_scanned = 0
    batches = 0

    cols = [COL_DONOR, COL_SEX, COL_ETH]

    for batch in pf.iter_batches(columns=cols, batch_size=BATCH_SIZE):
        batches += 1
        rows_scanned += batch.num_rows

        # Cast to string for robust handling (dictionary encoded columns are common in Arrow)
        donor = pc.cast(batch.column(0), pa.string())
        sex = pc.cast(batch.column(1), pa.string())
        eth = pc.cast(batch.column(2), pa.string())

        df = pa.Table.from_arrays([donor, sex, eth], names=cols).to_pandas()

        # Normalize values
        df[COL_DONOR] = df[COL_DONOR].map(_norm_str)
        df[COL_SEX] = df[COL_SEX].map(_norm_str)
        df[COL_ETH] = df[COL_ETH].map(_norm_str)

        # Drop rows without a meaningful donor_id (if any)
        df = df[df[COL_DONOR] != "unknown"]
        if df.empty:
            continue

        # Batch group-by to avoid per-row loops: (donor_id, sex, eth) -> cell count
        gb = df.groupby([COL_DONOR, COL_SEX, COL_ETH], dropna=False).size()

        for (d, s, e), n in gb.items():
            n = int(n)

            # cell-level counts
            cell_by_sex[s] += n
            cell_by_eth[e] += n
            cell_by_sex_eth[(s, e)] += n

            # donor-level sets and counts
            donors_all.add(d)
            donors_by_sex[s].add(d)
            donors_by_eth[e].add(d)
            donors_by_sex_eth[(s, e)].add(d)
            donor_cell_count[d] += n

            # metadata consistency check per donor
            if d in donor_sex and donor_sex[d] != s:
                donor_inconsistency[d] += 1
            else:
                donor_sex.setdefault(d, s)

            if d in donor_eth and donor_eth[d] != e:
                donor_inconsistency[d] += 1
            else:
                donor_eth.setdefault(d, e)

        if PROGRESS_EVERY and (batches % PROGRESS_EVERY == 0):
            print(f"[scan] batches={batches:,} rows_scanned={rows_scanned:,} unique_donors={len(donors_all):,}")

    # -----------------------------
    # Write exports (analogous to dev_stage_inventory)
    # -----------------------------
    total_unique_donors = len(donors_all)
    total_cells = rows_scanned  # should match pf.metadata.num_rows for a single-file parquet

    # 1) High-level summary
    summary = pd.DataFrame([{
        "parquet_path": str(PARQUET_PATH),
        "rows_scanned_cells": total_cells,
        "unique_donors": total_unique_donors,
        "unique_sex_labels": len(cell_by_sex),
        "unique_ethnicity_labels": len(cell_by_eth),
        "donors_with_conflicting_sex_or_ethnicity": sum(1 for d in donor_inconsistency if donor_inconsistency[d] > 0),
    }])
    summary.to_csv(OUTDIR / "metadata_summary.csv", index=False)

    # 2) Sex: cell counts + unique donors
    sex_rows = []
    for s, cell_n in cell_by_sex.most_common():
        sex_rows.append({
            "sex": s,
            "cell_count_in_parquet": cell_n,
            "unique_donors": len(donors_by_sex[s]),
        })
    pd.DataFrame(sex_rows).to_csv(OUTDIR / "sex_counts.csv", index=False)

    # 3) Ethnicity: cell counts + unique donors
    eth_rows = []
    for e, cell_n in cell_by_eth.most_common():
        eth_rows.append({
            "self_reported_ethnicity": e,
            "cell_count_in_parquet": cell_n,
            "unique_donors": len(donors_by_eth[e]),
        })
    pd.DataFrame(eth_rows).to_csv(OUTDIR / "ethnicity_counts.csv", index=False)

    # 4) Sex × Ethnicity: cell counts + unique donors
    se_rows = []
    for (s, e), cell_n in cell_by_sex_eth.most_common():
        se_rows.append({
            "sex": s,
            "self_reported_ethnicity": e,
            "cell_count_in_parquet": cell_n,
            "unique_donors": len(donors_by_sex_eth[(s, e)]),
        })
    pd.DataFrame(se_rows).to_csv(OUTDIR / "sex_by_ethnicity_counts.csv", index=False)

    # 5) Donor-level table (cells per donor + canonical sex/eth + inconsistency flag)
    donor_rows = []
    for d, cell_n in donor_cell_count.most_common():
        donor_rows.append({
            "donor_id": d,
            "cell_count_in_parquet": cell_n,
            "sex": donor_sex.get(d, "unknown"),
            "self_reported_ethnicity": donor_eth.get(d, "unknown"),
            "metadata_conflict_events": int(donor_inconsistency.get(d, 0)),
        })
    pd.DataFrame(donor_rows).to_csv(OUTDIR / "donor_level_counts.csv", index=False)

    print("\nDone.")
    print("Wrote exports to:")
    for f in [
        "metadata_summary.csv",
        "sex_counts.csv",
        "ethnicity_counts.csv",
        "sex_by_ethnicity_counts.csv",
        "donor_level_counts.csv",
    ]:
        print(" ", OUTDIR / f)


if __name__ == "__main__":
    main()


[scan] batches=10 rows_scanned=5,242,880 unique_donors=950
[scan] batches=20 rows_scanned=10,485,760 unique_donors=2,181
[scan] batches=30 rows_scanned=15,728,640 unique_donors=2,265
[scan] batches=40 rows_scanned=20,327,572 unique_donors=2,615

Done.
Wrote exports to:
  /home/crizza/census_exports/metadata_inventory/metadata_summary.csv
  /home/crizza/census_exports/metadata_inventory/sex_counts.csv
  /home/crizza/census_exports/metadata_inventory/ethnicity_counts.csv
  /home/crizza/census_exports/metadata_inventory/sex_by_ethnicity_counts.csv
  /home/crizza/census_exports/metadata_inventory/donor_level_counts.csv


In [1]:
from __future__ import annotations

from pathlib import Path
from collections import Counter, defaultdict
import re

import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt


PARQUET_PATH = Path("/home/crizza/census_exports/census_hsapiens_healthy_adult_obs.parquet")
OUTDIR = Path("/home/crizza/census_exports/figures_obs_summary")
OUTDIR.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 250_000

# Columns we’ll summarize (must exist in the parquet)
COLS_1D = ["tissue_general", "tissue", "assay", "cell_type", "sex", "development_stage"]
COLS_2D = [("tissue_general", "assay")]  # for a stacked-bar style summary


_year_old_re = re.compile(r"^\s*(\d+)\s*-\s*year-old stage\s*$", re.IGNORECASE)
_decade_re = re.compile(r"^\s*(first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth)\s+decade stage\s*$",
                        re.IGNORECASE)

def is_adult_stage_label(stage: str, min_age_years: int = 15) -> bool:
    """
    Best-effort adult>=min_age_years classifier from development_stage label.
    Keeps:
      - 'adult stage', 'young adult stage', 'prime adult stage', 'late adult stage', 'middle aged stage'
      - '{N}-year-old stage' where N>=min_age_years
      - '{X} decade stage' where decade >= second (>=10) is ambiguous; we keep >= second only if you want.
        Here we keep >= second decade conservatively; edit if you want stricter.
    Drops fetal/embryonic (weeks post-fertilization, LMP months), months-old, and unknown.
    """
    if stage is None:
        return False
    s = stage.strip().lower()
    if s == "" or s == "unknown":
        return False

    if "adult" in s or "middle aged" in s:
        return True

    m = _year_old_re.match(stage)
    if m:
        age = int(m.group(1))
        return age >= min_age_years

    m2 = _decade_re.match(stage)
    if m2:
        # Map decade words to starting age approx
        decade_map = {
            "first": 0, "second": 10, "third": 20, "fourth": 30, "fifth": 40,
            "sixth": 50, "seventh": 60, "eighth": 70, "ninth": 80, "tenth": 90
        }
        start_age = decade_map[m2.group(1).lower()]
        return start_age >= min_age_years

    # Months-old, weeks post-fertilization, LMP months, etc. -> treat as not adult
    return False


def plot_top_bar(counts: Counter, title: str, out_prefix: str, top_n: int = 25):
    items = counts.most_common(top_n)
    df = pd.DataFrame(items, columns=["label", "count"]).iloc[::-1]  # reverse for horizontal bar
    fig = plt.figure(figsize=(10, max(4, 0.25 * len(df))))
    plt.barh(df["label"], df["count"])
    plt.title(title)
    plt.xlabel("Cell count")
    plt.tight_layout()

    fig.savefig(OUTDIR / f"{out_prefix}.svg")
    fig.savefig(OUTDIR / f"{out_prefix}.png", dpi=300)
    plt.close(fig)

    df.to_csv(OUTDIR / f"{out_prefix}.csv", index=False)


def plot_stacked_bar(pair_counts: Counter, row_name: str, col_name: str,
                     title: str, out_prefix: str, top_rows: int = 15, top_cols: int = 6):
    """
    pair_counts keys: (row_value, col_value) -> count
    Produces stacked bars: top row categories, stacked by top col categories (others collapsed).
    """
    # Determine top rows/cols by marginal sums
    row_sum = Counter()
    col_sum = Counter()
    for (r, c), n in pair_counts.items():
        row_sum[r] += n
        col_sum[c] += n

    rows = [r for r, _ in row_sum.most_common(top_rows)]
    cols = [c for c, _ in col_sum.most_common(top_cols)]

    # Build matrix with an "Other" column
    mat = []
    for r in rows:
        row_total = 0
        row_vals = []
        for c in cols:
            v = pair_counts.get((r, c), 0)
            row_vals.append(v)
            row_total += v
        other = row_sum[r] - row_total
        row_vals.append(max(other, 0))
        mat.append(row_vals)

    col_labels = cols + ["Other"]
    df = pd.DataFrame(mat, index=rows, columns=col_labels)

    fig = plt.figure(figsize=(12, 6))
    bottom = None
    x = range(len(df.index))
    for j, cname in enumerate(df.columns):
        vals = df[cname].values
        if bottom is None:
            plt.bar(x, vals, label=cname)
            bottom = vals.astype(float)
        else:
            plt.bar(x, vals, bottom=bottom, label=cname)
            bottom = bottom + vals

    plt.xticks(list(x), df.index, rotation=45, ha="right")
    plt.ylabel("Cell count")
    plt.title(title)
    plt.legend(frameon=False, bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()

    fig.savefig(OUTDIR / f"{out_prefix}.svg")
    fig.savefig(OUTDIR / f"{out_prefix}.png", dpi=300)
    plt.close(fig)

    df.reset_index(names=row_name).to_csv(OUTDIR / f"{out_prefix}.csv", index=False)


def aggregate_counts(adult_only: bool = False, min_age_years: int = 15):
    pf = pq.ParquetFile(PARQUET_PATH)

    # Counters for 1D summaries
    counts_1d = {c: Counter() for c in COLS_1D}

    # Counters for 2D summaries
    counts_2d = {pair: Counter() for pair in COLS_2D}

    rows_seen = 0
    rows_kept = 0

    # Only load columns that exist
    available = set(pf.schema_arrow.names)
    cols_needed = [c for c in set(COLS_1D + [x for pair in COLS_2D for x in pair]) if c in available]

    for batch in pf.iter_batches(columns=cols_needed, batch_size=BATCH_SIZE):
        df = batch.to_pandas()
        rows_seen += len(df)

        if adult_only and "development_stage" in df.columns:
            mask = df["development_stage"].astype("string").apply(lambda s: is_adult_stage_label(s, min_age_years))
            df = df.loc[mask].copy()

        rows_kept += len(df)
        if len(df) == 0:
            continue

        # Update 1D counters
        for c in COLS_1D:
            if c in df.columns:
                counts_1d[c].update(df[c].dropna().astype(str).tolist())

        # Update 2D counters
        for (a, b) in COLS_2D:
            if a in df.columns and b in df.columns:
                pairs = zip(df[a].fillna("NA").astype(str), df[b].fillna("NA").astype(str))
                counts_2d[(a, b)].update(pairs)

        if rows_seen % (5_000_000) < BATCH_SIZE:
            print(f"Scanned {rows_seen:,} rows; kept {rows_kept:,} rows (adult_only={adult_only})")

    print(f"DONE. Total scanned={rows_seen:,}; total kept={rows_kept:,} (adult_only={adult_only})")
    return counts_1d, counts_2d


if __name__ == "__main__":
    # Toggle this:
    ADULT_ONLY = True   # set False if you want the full healthy-only composition

    counts_1d, counts_2d = aggregate_counts(adult_only=ADULT_ONLY, min_age_years=15)

    prefix = "adult15" if ADULT_ONLY else "all_stages"

    # 1D plots
    plot_top_bar(counts_1d["tissue_general"],
                 title=f"Healthy (normal) composition by tissue_general ({prefix})",
                 out_prefix=f"{prefix}_tissue_general_top25",
                 top_n=25)

    plot_top_bar(counts_1d["assay"],
                 title=f"Healthy (normal) composition by assay ({prefix})",
                 out_prefix=f"{prefix}_assay_top25",
                 top_n=25)

    plot_top_bar(counts_1d["cell_type"],
                 title=f"Healthy (normal) composition by cell_type ({prefix})",
                 out_prefix=f"{prefix}_cell_type_top25",
                 top_n=25)

    plot_top_bar(counts_1d["development_stage"],
                 title=f"Healthy (normal) composition by development_stage ({prefix})",
                 out_prefix=f"{prefix}_development_stage_top25",
                 top_n=25)

    plot_top_bar(counts_1d["sex"],
                 title=f"Healthy (normal) composition by sex ({prefix})",
                 out_prefix=f"{prefix}_sex",
                 top_n=10)

    # 2D stacked plot: assay composition within top tissues
    plot_stacked_bar(counts_2d[("tissue_general", "assay")],
                     row_name="tissue_general",
                     col_name="assay",
                     title=f"Assay composition within top tissues ({prefix})",
                     out_prefix=f"{prefix}_tissue_general_by_assay_stacked",
                     top_rows=15,
                     top_cols=6)

    print(f"Outputs written to: {OUTDIR}")


TypeError: Cannot setitem on a Categorical with a new category (NA), set the categories first

In [2]:
from __future__ import annotations

import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Sequence, Set

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import cellxgene_census as cxc


# -----------------------------
# Configuration
# -----------------------------
@dataclass(frozen=True)
class CensusConfig:
    census_version: str = "stable"
    organism: str = "homo_sapiens"


@dataclass(frozen=True)
class FiltersConfig:
    # Filters requested
    normal_disease_otid: str = "PATO:0000461"  # normal
    target_assay_labels: Sequence[str] = ("10x 3' v2", "10x 3' v3", "10x multiome")
    target_sexes: Sequence[str] = ("male", "female")
    target_suspension_type: str = "cell"
    min_age_years: int = 15


@dataclass(frozen=True)
class ExportConfig:
    outdir: Path = Path("/home/crizza/census_exports")
    outfile: str = "census_hsapiens_normal_cell_mf_10x_age15plus_obs_donor_ethnicity.parquet"
    overwrite: bool = True

    # Inventory CSVs directory (leave None to auto-detect)
    inventory_dir: Optional[Path] = None

    # Organ/tissue/cell-type metadata
    # - organ best default: tissue_general (coarse organ-level)
    organ_col: str = "tissue_general"
    tissue_cols: Sequence[str] = ("tissue", "tissue_general")
    cell_type_col: str = "cell_type"

    # Export behavior
    export_all_non_ontology: bool = True
    drop_ontology_term_id_cols: bool = True  # keep labels; drop *_ontology_term_id


CENSUS = CensusConfig()
FILTERS = FiltersConfig()
EXPORT = ExportConfig()

# Minimal required columns (fixed: removed the empty string)
BASE_REQUIRED_EXPORT_COLS: Sequence[str] = (
    "dataset_id",
    "donor_id",
    "self_reported_ethnicity",
)

# Explicitly require these “business” metadata fields too
REQUIRED_METADATA_COLS: Sequence[str] = (
    EXPORT.organ_col,         # organ-level (default: tissue_general)
    "tissue",                 # finer tissue
    EXPORT.cell_type_col,     # cell type label
)


# -----------------------------
# Helpers
# -----------------------------
def ensure_writable_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path}")


def resolve_outpath(outdir: Path, outfile: str, overwrite: bool) -> Path:
    ensure_writable_dir(outdir)
    outpath = outdir / outfile
    if outpath.exists():
        if overwrite:
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output exists and overwrite=False: {outpath}")
    return outpath


def _format_in_list(values: Sequence[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def detect_inventory_dir(user_hint: Optional[Path] = None) -> Path:
    """
    Locate the folder containing:
      - year_old_stage_pairs_counts.csv
      - development_stage_pairs_counts.csv
    """
    required = ["year_old_stage_pairs_counts.csv", "development_stage_pairs_counts.csv"]
    candidates: List[Path] = []
    if user_hint is not None:
        candidates.append(user_hint)

    candidates.extend(
        [
            Path("/home/crizza/census_exports/dev_stage_inventory"),
            Path.home() / "census_exports" / "dev_stage_inventory",
            Path("/mnt/data"),  # useful if running inside a notebook/chat environment
        ]
    )

    for d in candidates:
        if all((d / f).exists() for f in required):
            return d

    raise FileNotFoundError(
        "Could not find inventory CSVs. Set inventory_dir to the folder containing:\n"
        "  - year_old_stage_pairs_counts.csv\n"
        "  - development_stage_pairs_counts.csv"
    )


def build_stage_ids_ge15(inventory_dir: Path, min_age_years: int) -> List[str]:
    """
    Build development_stage_ontology_term_id allow-list for age >= min_age_years
    using your CSV inventories.
    """
    year_pairs = pd.read_csv(inventory_dir / "year_old_stage_pairs_counts.csv")
    pairs = pd.read_csv(inventory_dir / "development_stage_pairs_counts.csv")

    # Defensive typing
    year_pairs["age_years"] = pd.to_numeric(year_pairs["age_years"], errors="coerce")

    # 1) All exact N-year-old stages where N >= min_age_years
    ids: Set[str] = set(
        year_pairs.loc[year_pairs["age_years"] >= min_age_years, "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )

    # 2) Add adult bins/labels seen in your data
    extra_labels = [
        "15-19 year-old",
        "adult stage",
        "young adult stage",
        "prime adult stage",
        "late adult stage",
        "middle aged stage",
        "third decade stage",
        "fourth decade stage",
        "fifth decade stage",
        "sixth decade stage",
        "seventh decade stage",
        "eighth decade stage",
        "ninth decade stage",
        "60-79 year-old stage",
        "80 year-old and over stage",
        "90 year-old and over stage",
    ]

    extra_ids = (
        pairs.loc[pairs["development_stage"].isin(extra_labels), "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )
    ids.update(extra_ids)

    ids.discard("unknown")
    ids.discard("nan")
    return sorted(ids)


def lookup_assay_term_ids(census) -> Dict[str, str]:
    """
    Map assay label -> assay_ontology_term_id via census_info/summary_cell_counts.
    Avoids quoting issues with "10x 3' v3".
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )
    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(FILTERS.target_assay_labels))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in FILTERS.target_assay_labels if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for assay label(s): {missing}")
    return found


def build_obs_value_filter(census, inventory_dir: Path) -> str:
    """
    Combined filters:
      - is_primary_data
      - normal disease
      - male/female
      - suspension_type cell
      - assays: 10x v2/v3/multiome (via assay_ontology_term_id)
      - age >= min_age_years (via development_stage_ontology_term_id allow-list)
    """
    assay_map = lookup_assay_term_ids(census)
    assay_term_ids = [assay_map[x] for x in FILTERS.target_assay_labels]
    stage_ids = build_stage_ids_ge15(inventory_dir, FILTERS.min_age_years)

    vf_parts = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{FILTERS.normal_disease_otid}'",
        f"sex in {_format_in_list(FILTERS.target_sexes)}",
        f"suspension_type == '{FILTERS.target_suspension_type}'",
        f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}",
        f"development_stage_ontology_term_id in {_format_in_list(stage_ids)}",
    ]
    return " and ".join(vf_parts)


def _suggest_columns(schema_names: Sequence[str], needle: str, k: int = 25) -> List[str]:
    n = needle.lower()
    hits = [c for c in schema_names if n in c.lower()]
    return hits[:k]


def build_export_columns(schema_names: Sequence[str]) -> List[str]:
    """
    Export ALL non-ontology columns (optionally), and guarantee required columns exist.
    Drops any column containing 'ontology_term_id' from OUTPUT by default.
    """
    required_cols = list(BASE_REQUIRED_EXPORT_COLS) + list(REQUIRED_METADATA_COLS)

    missing = [c for c in required_cols if c not in schema_names]
    if missing:
        msg = "Required export column(s) not found in exp.obs schema:\n"
        for m in missing:
            msg += f"  - {m}\n"
            msg += f"    suggestions: {_suggest_columns(schema_names, m.split('_')[0])}\n"
        raise ValueError(msg)

    cols: List[str]
    if EXPORT.export_all_non_ontology:
        cols = list(schema_names)
        # Always drop soma_joinid from output unless you explicitly want it
        cols = [c for c in cols if c != "soma_joinid"]
        if EXPORT.drop_ontology_term_id_cols:
            cols = [c for c in cols if "ontology_term_id" not in c]
    else:
        # Only export the minimal set (plus whatever you add later)
        cols = list(dict.fromkeys(required_cols))  # preserve order, unique

    # Ensure required columns included
    for r in required_cols:
        if r not in cols:
            cols.append(r)

    return cols


def export_obs_to_parquet(exp, value_filter: str, outpath: Path, column_names: List[str]) -> None:
    """
    Stream obs rows and write to a single parquet.
    """
    it = exp.obs.read(value_filter=value_filter, column_names=column_names)

    writer: Optional[pq.ParquetWriter] = None
    rows = 0
    batches = 0

    try:
        for tbl in it:
            if isinstance(tbl, pa.RecordBatch):
                tbl = pa.Table.from_batches([tbl])
            elif not isinstance(tbl, pa.Table):
                tbl = pa.Table.from_batches(tbl.to_batches())

            if tbl.num_rows == 0:
                continue

            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema, compression="zstd")

            writer.write_table(tbl)

            rows += tbl.num_rows
            batches += 1
            if batches % 10 == 0:
                print(f"[export] batches={batches:,} rows={rows:,}")

    finally:
        if writer is not None:
            writer.close()

    print(f"Done. Wrote {rows:,} rows across {batches:,} batches to:\n  {outpath}")


# -----------------------------
# Main
# -----------------------------
def main() -> None:
    inventory_dir = detect_inventory_dir(EXPORT.inventory_dir)
    print("Using inventory CSV directory:", inventory_dir)

    outpath = resolve_outpath(EXPORT.outdir, EXPORT.outfile, EXPORT.overwrite)
    print("Output file:", outpath)

    with cxc.open_soma(census_version=CENSUS.census_version) as census:
        exp = census["census_data"][CENSUS.organism]

        value_filter = build_obs_value_filter(census, inventory_dir)
        print("\nCombined OBS value_filter:\n", value_filter, "\n")

        schema_names = list(exp.obs.schema.names)
        export_cols = build_export_columns(schema_names)

        print(f"Organ column (coarse): {EXPORT.organ_col}")
        print(f"Will export {len(export_cols)} obs columns "
              f"({'all non-ontology' if EXPORT.export_all_non_ontology else 'required only'}).")

        export_obs_to_parquet(exp, value_filter, outpath, export_cols)


if __name__ == "__main__":
    main()


Using inventory CSV directory: /home/crizza/census_exports/dev_stage_inventory
Output file: /home/crizza/census_exports/census_hsapiens_normal_cell_mf_10x_age15plus_obs_donor_ethnicity.parquet


The "stable" release is currently 2025-11-08. Specify 'census_version="2025-11-08"' in future calls to open_soma() to ensure data consistency.



Combined OBS value_filter:
 is_primary_data == True and disease_ontology_term_id == 'PATO:0000461' and sex in ['male', 'female'] and suspension_type == 'cell' and assay_ontology_term_id in ['EFO:0009899', 'EFO:0009922', 'EFO:0030059'] and development_stage_ontology_term_id in ['HsapDv:0000095', 'HsapDv:0000109', 'HsapDv:0000110', 'HsapDv:0000111', 'HsapDv:0000112', 'HsapDv:0000113', 'HsapDv:0000114', 'HsapDv:0000115', 'HsapDv:0000116', 'HsapDv:0000117', 'HsapDv:0000118', 'HsapDv:0000119', 'HsapDv:0000120', 'HsapDv:0000121', 'HsapDv:0000122', 'HsapDv:0000123', 'HsapDv:0000124', 'HsapDv:0000125', 'HsapDv:0000126', 'HsapDv:0000127', 'HsapDv:0000128', 'HsapDv:0000129', 'HsapDv:0000130', 'HsapDv:0000131', 'HsapDv:0000132', 'HsapDv:0000133', 'HsapDv:0000134', 'HsapDv:0000135', 'HsapDv:0000136', 'HsapDv:0000137', 'HsapDv:0000138', 'HsapDv:0000139', 'HsapDv:0000140', 'HsapDv:0000141', 'HsapDv:0000142', 'HsapDv:0000143', 'HsapDv:0000144', 'HsapDv:0000145', 'HsapDv:0000146', 'HsapDv:0000147', '

SOMAError: [ManagedQuery] [unnamed] Query FAILED: [TileDB::Task] Error: Caught std::exception: VFS: VFS parallel read error 's3://cellxgene-census-public-us-west-2/cell-census/2025-11-08/soma/census_data/homo_sapiens/obs/__fragments/__1762679146914_1762679146914_71bc71667c02c028dbb501720895e58e_22/a24.tdb'; Caught std::exception: S3: Failed to read S3 object s3://cellxgene-census-public-us-west-2/cell-census/2025-11-08/soma/census_data/homo_sapiens/obs/__fragments/__1762679146914_1762679146914_71bc71667c02c028dbb501720895e58e_22/a24.tdb[Error Type: 99] [HTTP Response Code: -1] : curlCode: 28, Timeout was reached; Details: Resolving timed out after 10800 milliseconds

In [1]:
from __future__ import annotations

import logging
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, Iterator, List, Optional, Sequence, Set

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import cellxgene_census as cxc

logger = logging.getLogger(__name__)


# -----------------------------
# Configuration
# -----------------------------
@dataclass(frozen=True, slots=True)
class CensusConfig:
    census_version: str = "stable"
    organism: str = "homo_sapiens"


@dataclass(frozen=True, slots=True)
class FiltersConfig:
    normal_disease_otid: str = "PATO:0000461"  # normal
    target_assay_labels: Sequence[str] = ("10x 3' v2", "10x 3' v3", "10x multiome")
    target_sexes: Sequence[str] = ("male", "female")
    target_suspension_type: str = "cell"
    min_age_years: Optional[int] = 15  # set None to disable age filter


@dataclass(frozen=True, slots=True)
class ExportConfig:
    outdir: Path = Path("./census_exports")
    outfile: str = "census_obs.parquet"
    overwrite: bool = True

    # Inventory CSVs directory (leave None to auto-detect)
    inventory_dir: Optional[Path] = None

    # Organ/tissue/cell-type metadata
    organ_col: str = "tissue_general"
    tissue_cols: Sequence[str] = ("tissue", "tissue_general")
    cell_type_col: str = "cell_type"

    # Export columns behavior
    export_all_columns: bool = False  # safer default than "all non-ontology"
    drop_ontology_term_id_cols: bool = False  # for atlas: keep key IDs by default

    # If dropping ontology IDs, allow-list the ones you still want (atlas-friendly)
    keep_ontology_term_id_cols: Sequence[str] = (
        "cell_type_ontology_term_id",
        "tissue_ontology_term_id",
        "assay_ontology_term_id",
        "disease_ontology_term_id",
        "development_stage_ontology_term_id",
    )

    # progress
    log_every_batches: int = 10


@dataclass(frozen=True, slots=True)
class CensusObsExportJob:
    census: CensusConfig = CensusConfig()
    filters: FiltersConfig = FiltersConfig()
    export: ExportConfig = ExportConfig()

    # minimal business columns you always want
    base_required_export_cols: Sequence[str] = (
        "dataset_id",
        "donor_id",
        "self_reported_ethnicity",
    )


# -----------------------------
# Filesystem helpers
# -----------------------------
def ensure_writable_dir(path: Path) -> None:
    path.mkdir(parents=True, exist_ok=True)
    if not os.access(path, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path}")


def resolve_outpath(outdir: Path, outfile: str, overwrite: bool) -> Path:
    ensure_writable_dir(outdir)
    outpath = outdir / outfile
    if outpath.exists():
        if overwrite:
            if not os.access(outpath, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {outpath}")
            outpath.unlink()
        else:
            raise FileExistsError(f"Output exists and overwrite=False: {outpath}")
    return outpath


# -----------------------------
# Census filter helpers
# -----------------------------
def _format_in_list(values: Sequence[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def detect_inventory_dir(user_hint: Optional[Path] = None) -> Path:
    """
    Locate folder containing:
      - year_old_stage_pairs_counts.csv
      - development_stage_pairs_counts.csv
    """
    required = ["year_old_stage_pairs_counts.csv", "development_stage_pairs_counts.csv"]
    candidates: List[Path] = []
    if user_hint is not None:
        candidates.append(user_hint)

    candidates.extend(
        [
            Path("./dev_stage_inventory"),
            Path.home() / "census_exports" / "dev_stage_inventory",
            Path("/mnt/data"),
        ]
    )

    for d in candidates:
        if all((d / f).exists() for f in required):
            return d

    raise FileNotFoundError(
        "Could not find inventory CSVs. Set export.inventory_dir to the folder containing:\n"
        "  - year_old_stage_pairs_counts.csv\n"
        "  - development_stage_pairs_counts.csv"
    )


def build_stage_ids_ge_min_age(inventory_dir: Path, min_age_years: int) -> List[str]:
    year_pairs = pd.read_csv(inventory_dir / "year_old_stage_pairs_counts.csv")
    pairs = pd.read_csv(inventory_dir / "development_stage_pairs_counts.csv")

    year_pairs["age_years"] = pd.to_numeric(year_pairs["age_years"], errors="coerce")

    ids: Set[str] = set(
        year_pairs.loc[year_pairs["age_years"] >= min_age_years, "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )

    extra_labels = [
        "15-19 year-old",
        "adult stage",
        "young adult stage",
        "prime adult stage",
        "late adult stage",
        "middle aged stage",
        "third decade stage",
        "fourth decade stage",
        "fifth decade stage",
        "sixth decade stage",
        "seventh decade stage",
        "eighth decade stage",
        "ninth decade stage",
        "60-79 year-old stage",
        "80 year-old and over stage",
        "90 year-old and over stage",
    ]
    extra_ids = (
        pairs.loc[pairs["development_stage"].isin(extra_labels), "development_stage_ontology_term_id"]
        .astype(str)
        .tolist()
    )
    ids.update(extra_ids)

    ids.discard("unknown")
    ids.discard("nan")
    return sorted(ids)


def lookup_assay_term_ids(census, assay_labels: Sequence[str]) -> Dict[str, str]:
    """
    Map assay label -> assay_ontology_term_id via census_info/summary_cell_counts.
    Avoids quoting issues with "10x 3' v3".
    """
    scc = (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )
    assay_rows = scc[(scc["category"] == "assay") & (scc["label"].isin(assay_labels))][
        ["label", "ontology_term_id"]
    ].drop_duplicates()

    found = dict(zip(assay_rows["label"], assay_rows["ontology_term_id"]))
    missing = [x for x in assay_labels if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for assay label(s): {missing}")
    return found


def build_obs_value_filter(
    census,
    filters: FiltersConfig,
    inventory_dir: Optional[Path],
) -> str:
    assay_map = lookup_assay_term_ids(census, filters.target_assay_labels)
    assay_term_ids = [assay_map[x] for x in filters.target_assay_labels]

    vf_parts = [
        "is_primary_data == True",
        f"disease_ontology_term_id == '{filters.normal_disease_otid}'",
        f"sex in {_format_in_list(filters.target_sexes)}",
        f"suspension_type == '{filters.target_suspension_type}'",
        f"assay_ontology_term_id in {_format_in_list(assay_term_ids)}",
    ]

    if filters.min_age_years is not None:
        if inventory_dir is None:
            raise ValueError("min_age_years is set, but inventory_dir is None.")
        stage_ids = build_stage_ids_ge_min_age(inventory_dir, filters.min_age_years)
        vf_parts.append(f"development_stage_ontology_term_id in {_format_in_list(stage_ids)}")

    return " and ".join(vf_parts)


# -----------------------------
# Column selection
# -----------------------------
def _suggest_columns(schema_names: Sequence[str], needle: str, k: int = 25) -> List[str]:
    n = needle.lower()
    hits = [c for c in schema_names if n in c.lower()]
    return hits[:k]


def build_export_columns(
    schema_names: Sequence[str],
    job: CensusObsExportJob,
) -> List[str]:
    exp_cfg = job.export

    required_cols = list(job.base_required_export_cols) + [
        exp_cfg.organ_col,
        "tissue",
        exp_cfg.cell_type_col,
    ]

    missing = [c for c in required_cols if c not in schema_names]
    if missing:
        msg = "Required export column(s) not found in exp.obs schema:\n"
        for m in missing:
            msg += f"  - {m}\n"
            msg += f"    suggestions: {_suggest_columns(schema_names, m.split('_')[0])}\n"
        raise ValueError(msg)

    if exp_cfg.export_all_columns:
        cols = [c for c in schema_names if c != "soma_joinid"]
    else:
        cols = list(dict.fromkeys(required_cols))

    # Handle ontology term ids
    if exp_cfg.drop_ontology_term_id_cols:
        keep = set(exp_cfg.keep_ontology_term_id_cols)
        cols = [c for c in cols if ("ontology_term_id" not in c) or (c in keep)]
        # if we kept some, ensure they exist
        for c in keep:
            if c in schema_names and c not in cols:
                cols.append(c)
    else:
        # Keep ontology IDs if they are already in cols (or if export_all_columns=True)
        pass

    # Ensure required columns included
    for r in required_cols:
        if r not in cols:
            cols.append(r)

    return cols


# -----------------------------
# Public API: stream or write parquet
# -----------------------------
def iter_obs_tables(job: CensusObsExportJob) -> Iterator[pa.Table]:
    """
    Stream obs as Arrow tables according to the job config.
    This is the “no-Parquet-required” path for downstream processing.
    """
    exp_cfg = job.export
    inventory_dir = None
    if job.filters.min_age_years is not None:
        inventory_dir = detect_inventory_dir(exp_cfg.inventory_dir)

    with cxc.open_soma(census_version=job.census.census_version) as census:
        exp = census["census_data"][job.census.organism]

        value_filter = build_obs_value_filter(census, job.filters, inventory_dir)
        schema_names = list(exp.obs.schema.names)
        export_cols = build_export_columns(schema_names, job)

        logger.info("OBS value_filter: %s", value_filter)
        logger.info("Exporting %d columns.", len(export_cols))

        it = exp.obs.read(value_filter=value_filter, column_names=export_cols)
        for tbl in it:
            if isinstance(tbl, pa.RecordBatch):
                tbl = pa.Table.from_batches([tbl])
            elif not isinstance(tbl, pa.Table):
                tbl = pa.Table.from_batches(tbl.to_batches())
            if tbl.num_rows:
                yield tbl


def export_obs_parquet(job: CensusObsExportJob) -> Path:
    """
    Write streamed obs rows to a single parquet file.
    """
    exp_cfg = job.export
    outpath = resolve_outpath(exp_cfg.outdir, exp_cfg.outfile, exp_cfg.overwrite)

    writer: Optional[pq.ParquetWriter] = None
    rows = 0
    batches = 0

    try:
        for tbl in iter_obs_tables(job):
            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema, compression="zstd")
            else:
                # Defensive: enforce consistent schema if something shifts unexpectedly
                if tbl.schema != writer.schema:
                    tbl = tbl.cast(writer.schema)

            writer.write_table(tbl)

            rows += tbl.num_rows
            batches += 1
            if exp_cfg.log_every_batches and (batches % exp_cfg.log_every_batches == 0):
                logger.info("export progress: batches=%s rows=%s", f"{batches:,}", f"{rows:,}")

    finally:
        if writer is not None:
            writer.close()

    logger.info("Done. Wrote %s rows across %s batches to %s", f"{rows:,}", f"{batches:,}", outpath)
    return outpath


In [ ]:
from __future__ import annotations

from dataclasses import dataclass, field
from pathlib import Path
from typing import Any, Dict, Iterable, Iterator, List, Literal, Optional, Sequence, Tuple, Union

import os

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import yaml

import cellxgene_census as cxc

try:
    import tiledbsoma as soma  # optional but recommended for TileDB context tuning
except Exception:  # pragma: no cover
    soma = None


# -----------------------------
# Schema helpers / constants
# -----------------------------
# These columns are shown in the official census query tutorial for obs keys. :contentReference[oaicite:2]{index=2}
OBS_TERM_ID_COLS: Dict[str, str] = {
    "assay": "assay_ontology_term_id",
    "cell_type": "cell_type_ontology_term_id",
    "development_stage": "development_stage_ontology_term_id",
    "disease": "disease_ontology_term_id",
    "self_reported_ethnicity": "self_reported_ethnicity_ontology_term_id",
    "sex": "sex_ontology_term_id",
    "tissue": "tissue_ontology_term_id",
    "tissue_general": "tissue_general_ontology_term_id",
}

OBS_LABEL_COLS: Dict[str, str] = {
    "assay": "assay",
    "cell_type": "cell_type",
    "development_stage": "development_stage",
    "disease": "disease",
    "self_reported_ethnicity": "self_reported_ethnicity",
    "sex": "sex",
    "tissue": "tissue",
    "tissue_general": "tissue_general",
}


# -----------------------------
# Config dataclasses
# -----------------------------
@dataclass(frozen=True)
class CensusTarget:
    census_version: str = "stable"
    organism: str = "homo_sapiens"
    measurement: str = "RNA"  # used for get_anndata


@dataclass(frozen=True)
class ObsFilters:
    # Canonical filters
    is_primary_data: Optional[bool] = True
    suspension_type: Optional[str] = "cell"

    # Prefer ontology term IDs where possible to avoid quoting/label instability
    disease_ontology_term_ids: List[str] = field(default_factory=list)
    disease_labels: List[str] = field(default_factory=list)

    sex_labels: List[str] = field(default_factory=list)
    sex_ontology_term_ids: List[str] = field(default_factory=list)

    assay_labels: List[str] = field(default_factory=list)
    assay_ontology_term_ids: List[str] = field(default_factory=list)

    # Tissue / organ
    tissue_general_labels: List[str] = field(default_factory=list)          # "organ" (coarse)
    tissue_general_ontology_term_ids: List[str] = field(default_factory=list)

    tissue_labels: List[str] = field(default_factory=list)                  # more specific
    tissue_ontology_term_ids: List[str] = field(default_factory=list)

    # Cell type
    cell_type_labels: List[str] = field(default_factory=list)
    cell_type_ontology_term_ids: List[str] = field(default_factory=list)

    # Development stage (optional)
    development_stage_labels: List[str] = field(default_factory=list)
    development_stage_ontology_term_ids: List[str] = field(default_factory=list)

    # Any additional value_filter fragment (appended with "and")
    extra_value_filter: Optional[str] = None


@dataclass(frozen=True)
class OutputSpec:
    mode: Literal["pandas", "arrow", "anndata", "parquet"] = "pandas"
    outpath: Optional[str] = None
    overwrite: bool = True
    parquet_compression: str = "zstd"


@dataclass(frozen=True)
class QuerySpec:
    target: CensusTarget = field(default_factory=CensusTarget)
    obs_filters: ObsFilters = field(default_factory=ObsFilters)

    # What to return / export
    export_all_non_ontology_obs_columns: bool = False
    obs_columns: List[str] = field(default_factory=list)  # if empty, use a sensible default

    # Only used for mode="anndata"
    var_value_filter: Optional[str] = None
    var_columns: List[str] = field(default_factory=list)

    # Optional TileDB config to mitigate S3 timeouts (see Output below)
    tiledb_config: Dict[str, Any] = field(default_factory=dict)

    output: OutputSpec = field(default_factory=OutputSpec)


# -----------------------------
# YAML loading
# -----------------------------
def load_query_spec_yaml(path: Union[str, Path]) -> QuerySpec:
    path = Path(path)
    with path.open("r", encoding="utf-8") as f:
        raw = yaml.safe_load(f)

    # minimal manual parsing (keeps dependencies light)
    target = CensusTarget(**raw.get("target", {}))
    obs_filters = ObsFilters(**raw.get("obs_filters", {}))
    output = OutputSpec(**raw.get("output", {}))

    return QuerySpec(
        target=target,
        obs_filters=obs_filters,
        export_all_non_ontology_obs_columns=raw.get("export_all_non_ontology_obs_columns", False),
        obs_columns=raw.get("obs_columns", []) or [],
        var_value_filter=raw.get("var_value_filter"),
        var_columns=raw.get("var_columns", []) or [],
        tiledb_config=raw.get("tiledb_config", {}) or {},
        output=output,
    )


# -----------------------------
# Core helpers
# -----------------------------
def _format_in_list(values: Sequence[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def _get_scc_table(census) -> pd.DataFrame:
    """Load census_info.summary_cell_counts as a pandas table."""
    return (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )


def resolve_ontology_term_ids(
    census,
    category: str,
    labels: Sequence[str],
) -> List[str]:
    """
    Resolve label -> ontology_term_id for a given summary_cell_counts category.
    """
    if not labels:
        return []
    scc = _get_scc_table(census)
    sub = scc[(scc["category"] == category) & (scc["label"].isin(list(labels)))]
    found = dict(zip(sub["label"], sub["ontology_term_id"]))
    missing = [x for x in labels if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for {category} label(s): {missing}")
    return [found[x] for x in labels]


def choose_organ_column(obs_keys: Sequence[str]) -> str:
    """
    In Census obs, the typical coarse “organ” column is `tissue_general`.
    If absent, fall back to `tissue`. :contentReference[oaicite:3]{index=3}
    """
    if "tissue_general" in obs_keys:
        return "tissue_general"
    if "tissue" in obs_keys:
        return "tissue"
    raise ValueError("Neither tissue_general nor tissue found in obs schema keys.")


def build_obs_value_filter(census, obs_filters: ObsFilters) -> str:
    """
    Build a Census-compatible obs value_filter string.
    Prefers ontology-term-id columns when possible.
    """
    parts: List[str] = []

    # is_primary_data / suspension_type (columns are present in obs keys) :contentReference[oaicite:4]{index=4}
    if obs_filters.is_primary_data is not None:
        parts.append(f"is_primary_data == {bool(obs_filters.is_primary_data)}")
    if obs_filters.suspension_type:
        parts.append(f"suspension_type == '{obs_filters.suspension_type}'")

    # disease
    disease_ids = list(obs_filters.disease_ontology_term_ids)
    if not disease_ids and obs_filters.disease_labels:
        disease_ids = resolve_ontology_term_ids(census, "disease", obs_filters.disease_labels)
    if disease_ids:
        parts.append(f"{OBS_TERM_ID_COLS['disease']} in {_format_in_list(disease_ids)}")

    # sex
    sex_ids = list(obs_filters.sex_ontology_term_ids)
    if not sex_ids and obs_filters.sex_labels:
        # Either filter by label, or resolve to ontology ids. Prefer ids.
        sex_ids = resolve_ontology_term_ids(census, "sex", obs_filters.sex_labels)
    if sex_ids:
        parts.append(f"{OBS_TERM_ID_COLS['sex']} in {_format_in_list(sex_ids)}")

    # assay
    assay_ids = list(obs_filters.assay_ontology_term_ids)
    if not assay_ids and obs_filters.assay_labels:
        assay_ids = resolve_ontology_term_ids(census, "assay", obs_filters.assay_labels)
    if assay_ids:
        parts.append(f"{OBS_TERM_ID_COLS['assay']} in {_format_in_list(assay_ids)}")

    # tissue_general (organ/coarse)
    tg_ids = list(obs_filters.tissue_general_ontology_term_ids)
    if not tg_ids and obs_filters.tissue_general_labels:
        tg_ids = resolve_ontology_term_ids(census, "tissue_general", obs_filters.tissue_general_labels)
    if tg_ids:
        parts.append(f"{OBS_TERM_ID_COLS['tissue_general']} in {_format_in_list(tg_ids)}")

    # tissue (fine)
    tissue_ids = list(obs_filters.tissue_ontology_term_ids)
    if not tissue_ids and obs_filters.tissue_labels:
        tissue_ids = resolve_ontology_term_ids(census, "tissue", obs_filters.tissue_labels)
    if tissue_ids:
        parts.append(f"{OBS_TERM_ID_COLS['tissue']} in {_format_in_list(tissue_ids)}")

    # cell_type
    ct_ids = list(obs_filters.cell_type_ontology_term_ids)
    if not ct_ids and obs_filters.cell_type_labels:
        ct_ids = resolve_ontology_term_ids(census, "cell_type", obs_filters.cell_type_labels)
    if ct_ids:
        parts.append(f"{OBS_TERM_ID_COLS['cell_type']} in {_format_in_list(ct_ids)}")

    # development_stage (optional)
    ds_ids = list(obs_filters.development_stage_ontology_term_ids)
    if not ds_ids and obs_filters.development_stage_labels:
        ds_ids = resolve_ontology_term_ids(census, "development_stage", obs_filters.development_stage_labels)
    if ds_ids:
        parts.append(f"{OBS_TERM_ID_COLS['development_stage']} in {_format_in_list(ds_ids)}")

    # extra fragment
    if obs_filters.extra_value_filter:
        parts.append(f"({obs_filters.extra_value_filter})")

    return " and ".join(parts) if parts else ""


def _default_obs_columns() -> List[str]:
    # Includes tissue + coarse organ + cell type labels; excludes *_ontology_term_id by default.
    return [
        "dataset_id",
        "donor_id",
        "assay",
        "cell_type",
        "tissue",
        "tissue_general",
        "disease",
        "development_stage",
        "sex",
        "self_reported_ethnicity",
        "is_primary_data",
        "suspension_type",
    ]


def build_obs_export_columns(obs_keys: Sequence[str], export_all_non_ontology: bool, user_cols: Sequence[str]) -> List[str]:
    keys = list(obs_keys)

    if export_all_non_ontology:
        cols = [c for c in keys if (c != "soma_joinid" and "ontology_term_id" not in c)]
        return cols

    cols = list(user_cols) if user_cols else _default_obs_columns()
    missing = [c for c in cols if c not in keys]
    if missing:
        raise ValueError(f"Requested obs columns not present in schema: {missing}")
    return cols


# -----------------------------
# IO / execution
# -----------------------------
def _ensure_writable_parent(path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    if not os.access(path.parent, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path.parent}")


def _resolve_outpath(outpath: str, overwrite: bool) -> Path:
    p = Path(outpath)
    _ensure_writable_parent(p)
    if p.exists():
        if overwrite:
            if not os.access(p, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {p}")
            p.unlink()
        else:
            raise FileExistsError(f"Output exists and overwrite=False: {p}")
    return p


def _make_soma_context(tiledb_config: Dict[str, Any]):
    """
    Optional: create a SOMATileDBContext to tune S3 timeouts/retries/parallel ops.
    """
    if not tiledb_config:
        return None
    if soma is None:
        raise RuntimeError("tiledbsoma is not importable, but tiledb_config was provided.")
    return soma.SOMATileDBContext(tiledb_config=tiledb_config)


def stream_obs_tables(
    exp,
    value_filter: str,
    column_names: Sequence[str],
) -> Iterator[pa.Table]:
    """
    Stream obs results as Arrow Tables.
    """
    it = exp.obs.read(value_filter=value_filter, column_names=list(column_names))
    for tbl in it:
        if isinstance(tbl, pa.RecordBatch):
            yield pa.Table.from_batches([tbl])
        elif isinstance(tbl, pa.Table):
            yield tbl
        else:
            yield pa.Table.from_batches(tbl.to_batches())


def write_parquet_stream(
    tables: Iterable[pa.Table],
    outpath: Path,
    compression: str = "zstd",
) -> Tuple[int, int]:
    writer: Optional[pq.ParquetWriter] = None
    rows = 0
    batches = 0
    try:
        for tbl in tables:
            if tbl.num_rows == 0:
                continue
            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema, compression=compression)
            writer.write_table(tbl)
            rows += tbl.num_rows
            batches += 1
    finally:
        if writer is not None:
            writer.close()
    return rows, batches


def run_query(spec: QuerySpec) -> Union[pd.DataFrame, pa.Table, Any, Path]:
    """
    Execute the query.
    Returns:
      - pandas DataFrame (mode=pandas)
      - Arrow Table (mode=arrow; concatenated, so only for moderate result sizes)
      - AnnData (mode=anndata) via cellxgene_census.get_anndata :contentReference[oaicite:5]{index=5}
      - Path to parquet (mode=parquet)
    """
    context = _make_soma_context(spec.tiledb_config)
    with cxc.open_soma(census_version=spec.target.census_version, context=context) as census:
        exp = census["census_data"][spec.target.organism]

        obs_keys = list(exp.obs.keys())
        organ_col = choose_organ_column(obs_keys)  # tissue_general preferred :contentReference[oaicite:6]{index=6}

        value_filter = build_obs_value_filter(census, spec.obs_filters)

        # Build export columns; ensure organ/tissue + cell_type available
        export_cols = build_obs_export_columns(
            obs_keys=obs_keys,
            export_all_non_ontology=spec.export_all_non_ontology_obs_columns,
            user_cols=spec.obs_columns,
        )
        # Ensure core metadata are present
        for required in ("cell_type", "tissue", organ_col):
            if required in obs_keys and required not in export_cols:
                export_cols.append(required)

        mode = spec.output.mode

        if mode == "anndata":
            # get_anndata is explicitly documented as the convenient API for expression+metadata slices. :contentReference[oaicite:7]{index=7}
            column_names = {"obs": export_cols}
            if spec.var_columns:
                column_names["var"] = spec.var_columns

            adata = cxc.get_anndata(
                census=census,
                organism="Homo sapiens" if spec.target.organism == "homo_sapiens" else "Mus musculus",
                obs_value_filter=value_filter or None,
                var_value_filter=spec.var_value_filter or None,
                column_names=column_names,
            )
            return adata

        if mode == "pandas":
            df = exp.obs.read(value_filter=value_filter or None, column_names=export_cols).concat().to_pandas()
            return df

        if mode == "arrow":
            tables = list(stream_obs_tables(exp, value_filter=value_filter, column_names=export_cols))
            if not tables:
                return pa.table({c: [] for c in export_cols})
            return pa.concat_tables(tables, promote=True)

        if mode == "parquet":
            if not spec.output.outpath:
                raise ValueError("output.outpath is required for mode='parquet'")
            outpath = _resolve_outpath(spec.output.outpath, spec.output.overwrite)

            tables = stream_obs_tables(exp, value_filter=value_filter, column_names=export_cols)
            rows, batches = write_parquet_stream(tables, outpath, compression=spec.output.parquet_compression)
            # Optionally you could log rows/batches here; returning the path keeps this a clean module API.
            return outpath

        raise ValueError(f"Unknown output mode: {mode}")


ModuleNotFoundError: No module named 'cellstrata'

In [4]:
from __future__ import annotations

from dataclasses import dataclass, field
from pathlib import Path
from typing import Any, Dict, Iterable, Iterator, List, Literal, Optional, Sequence, Tuple, Union

import os

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import yaml

import cellxgene_census as cxc

try:
    import tiledbsoma as soma  # optional but recommended for TileDB context tuning
except Exception:  # pragma: no cover
    soma = None


# -----------------------------
# Schema helpers / constants
# -----------------------------
# These columns are shown in the official census query tutorial for obs keys. :contentReference[oaicite:2]{index=2}
OBS_TERM_ID_COLS: Dict[str, str] = {
    "assay": "assay_ontology_term_id",
    "cell_type": "cell_type_ontology_term_id",
    "development_stage": "development_stage_ontology_term_id",
    "disease": "disease_ontology_term_id",
    "self_reported_ethnicity": "self_reported_ethnicity_ontology_term_id",
    "sex": "sex_ontology_term_id",
    "tissue": "tissue_ontology_term_id",
    "tissue_general": "tissue_general_ontology_term_id",
}

OBS_LABEL_COLS: Dict[str, str] = {
    "assay": "assay",
    "cell_type": "cell_type",
    "development_stage": "development_stage",
    "disease": "disease",
    "self_reported_ethnicity": "self_reported_ethnicity",
    "sex": "sex",
    "tissue": "tissue",
    "tissue_general": "tissue_general",
}


# -----------------------------
# Config dataclasses
# -----------------------------
@dataclass(frozen=True)
class CensusTarget:
    census_version: str = "stable"
    organism: str = "homo_sapiens"
    measurement: str = "RNA"  # used for get_anndata


@dataclass(frozen=True)
class ObsFilters:
    # Canonical filters
    is_primary_data: Optional[bool] = True
    suspension_type: Optional[str] = "cell"

    # Prefer ontology term IDs where possible to avoid quoting/label instability
    disease_ontology_term_ids: List[str] = field(default_factory=list)
    disease_labels: List[str] = field(default_factory=list)

    sex_labels: List[str] = field(default_factory=list)
    sex_ontology_term_ids: List[str] = field(default_factory=list)

    assay_labels: List[str] = field(default_factory=list)
    assay_ontology_term_ids: List[str] = field(default_factory=list)

    # Tissue / organ
    tissue_general_labels: List[str] = field(default_factory=list)          # "organ" (coarse)
    tissue_general_ontology_term_ids: List[str] = field(default_factory=list)

    tissue_labels: List[str] = field(default_factory=list)                  # more specific
    tissue_ontology_term_ids: List[str] = field(default_factory=list)

    # Cell type
    cell_type_labels: List[str] = field(default_factory=list)
    cell_type_ontology_term_ids: List[str] = field(default_factory=list)

    # Development stage (optional)
    development_stage_labels: List[str] = field(default_factory=list)
    development_stage_ontology_term_ids: List[str] = field(default_factory=list)

    # Any additional value_filter fragment (appended with "and")
    extra_value_filter: Optional[str] = None


@dataclass(frozen=True)
class OutputSpec:
    mode: Literal["pandas", "arrow", "anndata", "parquet"] = "pandas"
    outpath: Optional[str] = None
    overwrite: bool = True
    parquet_compression: str = "zstd"


@dataclass(frozen=True)
class QuerySpec:
    target: CensusTarget = field(default_factory=CensusTarget)
    obs_filters: ObsFilters = field(default_factory=ObsFilters)

    # What to return / export
    export_all_non_ontology_obs_columns: bool = False
    obs_columns: List[str] = field(default_factory=list)  # if empty, use a sensible default

    # Only used for mode="anndata"
    var_value_filter: Optional[str] = None
    var_columns: List[str] = field(default_factory=list)

    # Optional TileDB config to mitigate S3 timeouts (see Output below)
    tiledb_config: Dict[str, Any] = field(default_factory=dict)

    output: OutputSpec = field(default_factory=OutputSpec)


# -----------------------------
# YAML loading
# -----------------------------
def load_query_spec_yaml(path: Union[str, Path]) -> QuerySpec:
    path = Path(path)
    with path.open("r", encoding="utf-8") as f:
        raw = yaml.safe_load(f)

    # minimal manual parsing (keeps dependencies light)
    target = CensusTarget(**raw.get("target", {}))
    obs_filters = ObsFilters(**raw.get("obs_filters", {}))
    output = OutputSpec(**raw.get("output", {}))

    return QuerySpec(
        target=target,
        obs_filters=obs_filters,
        export_all_non_ontology_obs_columns=raw.get("export_all_non_ontology_obs_columns", False),
        obs_columns=raw.get("obs_columns", []) or [],
        var_value_filter=raw.get("var_value_filter"),
        var_columns=raw.get("var_columns", []) or [],
        tiledb_config=raw.get("tiledb_config", {}) or {},
        output=output,
    )


# -----------------------------
# Core helpers
# -----------------------------
def _format_in_list(values: Sequence[str]) -> str:
    """SOMA value_filter list literal: ['a','b'] with minimal escaping."""
    esc = [v.replace("\\", "\\\\").replace("'", "\\'") for v in values]
    return "[" + ", ".join([f"'{v}'" for v in esc]) + "]"


def _get_scc_table(census) -> pd.DataFrame:
    """Load census_info.summary_cell_counts as a pandas table."""
    return (
        census["census_info"]["summary_cell_counts"]
        .read(column_names=["category", "label", "ontology_term_id"])
        .concat()
        .to_pandas()
    )


def resolve_ontology_term_ids(
    census,
    category: str,
    labels: Sequence[str],
) -> List[str]:
    """
    Resolve label -> ontology_term_id for a given summary_cell_counts category.
    """
    if not labels:
        return []
    scc = _get_scc_table(census)
    sub = scc[(scc["category"] == category) & (scc["label"].isin(list(labels)))]
    found = dict(zip(sub["label"], sub["ontology_term_id"]))
    missing = [x for x in labels if x not in found]
    if missing:
        raise ValueError(f"Could not resolve ontology_term_id for {category} label(s): {missing}")
    return [found[x] for x in labels]


def choose_organ_column(obs_keys: Sequence[str]) -> str:
    """
    In Census obs, the typical coarse “organ” column is `tissue_general`.
    If absent, fall back to `tissue`. :contentReference[oaicite:3]{index=3}
    """
    if "tissue_general" in obs_keys:
        return "tissue_general"
    if "tissue" in obs_keys:
        return "tissue"
    raise ValueError("Neither tissue_general nor tissue found in obs schema keys.")


def build_obs_value_filter(census, obs_filters: ObsFilters) -> str:
    """
    Build a Census-compatible obs value_filter string.
    Prefers ontology-term-id columns when possible.
    """
    parts: List[str] = []

    # is_primary_data / suspension_type (columns are present in obs keys) :contentReference[oaicite:4]{index=4}
    if obs_filters.is_primary_data is not None:
        parts.append(f"is_primary_data == {bool(obs_filters.is_primary_data)}")
    if obs_filters.suspension_type:
        parts.append(f"suspension_type == '{obs_filters.suspension_type}'")

    # disease
    disease_ids = list(obs_filters.disease_ontology_term_ids)
    if not disease_ids and obs_filters.disease_labels:
        disease_ids = resolve_ontology_term_ids(census, "disease", obs_filters.disease_labels)
    if disease_ids:
        parts.append(f"{OBS_TERM_ID_COLS['disease']} in {_format_in_list(disease_ids)}")

    # sex
    sex_ids = list(obs_filters.sex_ontology_term_ids)
    if not sex_ids and obs_filters.sex_labels:
        # Either filter by label, or resolve to ontology ids. Prefer ids.
        sex_ids = resolve_ontology_term_ids(census, "sex", obs_filters.sex_labels)
    if sex_ids:
        parts.append(f"{OBS_TERM_ID_COLS['sex']} in {_format_in_list(sex_ids)}")

    # assay
    assay_ids = list(obs_filters.assay_ontology_term_ids)
    if not assay_ids and obs_filters.assay_labels:
        assay_ids = resolve_ontology_term_ids(census, "assay", obs_filters.assay_labels)
    if assay_ids:
        parts.append(f"{OBS_TERM_ID_COLS['assay']} in {_format_in_list(assay_ids)}")

    # tissue_general (organ/coarse)
    tg_ids = list(obs_filters.tissue_general_ontology_term_ids)
    if not tg_ids and obs_filters.tissue_general_labels:
        tg_ids = resolve_ontology_term_ids(census, "tissue_general", obs_filters.tissue_general_labels)
    if tg_ids:
        parts.append(f"{OBS_TERM_ID_COLS['tissue_general']} in {_format_in_list(tg_ids)}")

    # tissue (fine)
    tissue_ids = list(obs_filters.tissue_ontology_term_ids)
    if not tissue_ids and obs_filters.tissue_labels:
        tissue_ids = resolve_ontology_term_ids(census, "tissue", obs_filters.tissue_labels)
    if tissue_ids:
        parts.append(f"{OBS_TERM_ID_COLS['tissue']} in {_format_in_list(tissue_ids)}")

    # cell_type
    ct_ids = list(obs_filters.cell_type_ontology_term_ids)
    if not ct_ids and obs_filters.cell_type_labels:
        ct_ids = resolve_ontology_term_ids(census, "cell_type", obs_filters.cell_type_labels)
    if ct_ids:
        parts.append(f"{OBS_TERM_ID_COLS['cell_type']} in {_format_in_list(ct_ids)}")

    # development_stage (optional)
    ds_ids = list(obs_filters.development_stage_ontology_term_ids)
    if not ds_ids and obs_filters.development_stage_labels:
        ds_ids = resolve_ontology_term_ids(census, "development_stage", obs_filters.development_stage_labels)
    if ds_ids:
        parts.append(f"{OBS_TERM_ID_COLS['development_stage']} in {_format_in_list(ds_ids)}")

    # extra fragment
    if obs_filters.extra_value_filter:
        parts.append(f"({obs_filters.extra_value_filter})")

    return " and ".join(parts) if parts else ""


def _default_obs_columns() -> List[str]:
    # Includes tissue + coarse organ + cell type labels; excludes *_ontology_term_id by default.
    return [
        "dataset_id",
        "donor_id",
        "assay",
        "cell_type",
        "tissue",
        "tissue_general",
        "disease",
        "development_stage",
        "sex",
        "self_reported_ethnicity",
        "is_primary_data",
        "suspension_type",
    ]


def build_obs_export_columns(obs_keys: Sequence[str], export_all_non_ontology: bool, user_cols: Sequence[str]) -> List[str]:
    keys = list(obs_keys)

    if export_all_non_ontology:
        cols = [c for c in keys if (c != "soma_joinid" and "ontology_term_id" not in c)]
        return cols

    cols = list(user_cols) if user_cols else _default_obs_columns()
    missing = [c for c in cols if c not in keys]
    if missing:
        raise ValueError(f"Requested obs columns not present in schema: {missing}")
    return cols


# -----------------------------
# IO / execution
# -----------------------------
def _ensure_writable_parent(path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    if not os.access(path.parent, os.W_OK):
        raise PermissionError(f"Output directory is not writable: {path.parent}")


def _resolve_outpath(outpath: str, overwrite: bool) -> Path:
    p = Path(outpath)
    _ensure_writable_parent(p)
    if p.exists():
        if overwrite:
            if not os.access(p, os.W_OK):
                raise PermissionError(f"Cannot overwrite existing file: {p}")
            p.unlink()
        else:
            raise FileExistsError(f"Output exists and overwrite=False: {p}")
    return p


def _make_soma_context(tiledb_config: Dict[str, Any]):
    """
    Optional: create a SOMATileDBContext to tune S3 timeouts/retries/parallel ops.
    """
    if not tiledb_config:
        return None
    if soma is None:
        raise RuntimeError("tiledbsoma is not importable, but tiledb_config was provided.")
    return soma.SOMATileDBContext(tiledb_config=tiledb_config)


def stream_obs_tables(
    exp,
    value_filter: str,
    column_names: Sequence[str],
) -> Iterator[pa.Table]:
    """
    Stream obs results as Arrow Tables.
    """
    it = exp.obs.read(value_filter=value_filter, column_names=list(column_names))
    for tbl in it:
        if isinstance(tbl, pa.RecordBatch):
            yield pa.Table.from_batches([tbl])
        elif isinstance(tbl, pa.Table):
            yield tbl
        else:
            yield pa.Table.from_batches(tbl.to_batches())


def write_parquet_stream(
    tables: Iterable[pa.Table],
    outpath: Path,
    compression: str = "zstd",
) -> Tuple[int, int]:
    writer: Optional[pq.ParquetWriter] = None
    rows = 0
    batches = 0
    try:
        for tbl in tables:
            if tbl.num_rows == 0:
                continue
            if writer is None:
                writer = pq.ParquetWriter(str(outpath), tbl.schema, compression=compression)
            writer.write_table(tbl)
            rows += tbl.num_rows
            batches += 1
    finally:
        if writer is not None:
            writer.close()
    return rows, batches


def run_query(spec: QuerySpec) -> Union[pd.DataFrame, pa.Table, Any, Path]:
    """
    Execute the query.
    Returns:
      - pandas DataFrame (mode=pandas)
      - Arrow Table (mode=arrow; concatenated, so only for moderate result sizes)
      - AnnData (mode=anndata) via cellxgene_census.get_anndata :contentReference[oaicite:5]{index=5}
      - Path to parquet (mode=parquet)
    """
    context = _make_soma_context(spec.tiledb_config)
    with cxc.open_soma(census_version=spec.target.census_version, context=context) as census:
        exp = census["census_data"][spec.target.organism]

        obs_keys = list(exp.obs.keys())
        organ_col = choose_organ_column(obs_keys)  # tissue_general preferred :contentReference[oaicite:6]{index=6}

        value_filter = build_obs_value_filter(census, spec.obs_filters)

        # Build export columns; ensure organ/tissue + cell_type available
        export_cols = build_obs_export_columns(
            obs_keys=obs_keys,
            export_all_non_ontology=spec.export_all_non_ontology_obs_columns,
            user_cols=spec.obs_columns,
        )
        # Ensure core metadata are present
        for required in ("cell_type", "tissue", organ_col):
            if required in obs_keys and required not in export_cols:
                export_cols.append(required)

        mode = spec.output.mode

        if mode == "anndata":
            # get_anndata is explicitly documented as the convenient API for expression+metadata slices. :contentReference[oaicite:7]{index=7}
            column_names = {"obs": export_cols}
            if spec.var_columns:
                column_names["var"] = spec.var_columns

            adata = cxc.get_anndata(
                census=census,
                organism="Homo sapiens" if spec.target.organism == "homo_sapiens" else "Mus musculus",
                obs_value_filter=value_filter or None,
                var_value_filter=spec.var_value_filter or None,
                column_names=column_names,
            )
            return adata

        if mode == "pandas":
            df = exp.obs.read(value_filter=value_filter or None, column_names=export_cols).concat().to_pandas()
            return df

        if mode == "arrow":
            tables = list(stream_obs_tables(exp, value_filter=value_filter, column_names=export_cols))
            if not tables:
                return pa.table({c: [] for c in export_cols})
            return pa.concat_tables(tables, promote=True)

        if mode == "parquet":
            if not spec.output.outpath:
                raise ValueError("output.outpath is required for mode='parquet'")
            outpath = _resolve_outpath(spec.output.outpath, spec.output.overwrite)

            tables = stream_obs_tables(exp, value_filter=value_filter, column_names=export_cols)
            rows, batches = write_parquet_stream(tables, outpath, compression=spec.output.parquet_compression)
            # Optionally you could log rows/batches here; returning the path keeps this a clean module API.
            return outpath

        raise ValueError(f"Unknown output mode: {mode}")
